<a href="https://colab.research.google.com/github/Uzumaki-Naruto07/Data_Vis/blob/main/muqabala_colab_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1: SETUP AND IMPORTS

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (697 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 124947 files and directories currently installed.)
Pre

In [ ]:
!pip install transformers torch huggingface_hub gtts pandas numpy scikit-learn joblib SpeechRecognition pyaudio tensorflow matplotlib datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.5 MB/s 

In [ ]:
# ============== SECTION 1: SETUP AND IMPORTS ==============
# Install required packages with pip:
# pip install transformers torch huggingface_hub gtts pandas numpy scikit-learn joblib SpeechRecognition pyaudio tensorflow matplotlib datasets torch

# Initialize Colab detection variable at the very beginning
USING_COLAB = False


import os
import json
import time
import re
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from getpass import getpass
import joblib
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter


# Initialize feature flags
TTS_AVAILABLE = False
SPEECH_RECOGNITION_AVAILABLE = False
ML_AVAILABLE = False
TF_AVAILABLE = False
XGBOOST_AVAILABLE = False
TRANSFORMERS_AVAILABLE = False
SENTIMENT_ANALYSIS_AVAILABLE = False
DATASETS_AVAILABLE = False

# Try importing optional packages
try:
    from sklearn.linear_model import LinearRegression
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
    from sklearn.model_selection import train_test_split, cross_val_score
    from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
    from sklearn.preprocessing import StandardScaler
    import joblib
    import matplotlib.pyplot as plt
    ML_AVAILABLE = True
    print("✅ Machine learning support available")
except ImportError:
    print("❌ Machine learning libraries not available")

try:
    # TensorFlow is optional for neural network models
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout
    from tensorflow.keras.optimizers import Adam
    TF_AVAILABLE = True
    print("✅ TensorFlow/Keras support available for neural networks")
except ImportError:
    print("❌ TensorFlow not available - advanced ML models will be limited")

try:
    from gtts import gTTS
    import os
    TTS_AVAILABLE = True
    print("✅ Text-to-speech support available")
except ImportError:
    print("❌ Text-to-speech not available")

try:
    import speech_recognition as sr
    SPEECH_RECOGNITION_AVAILABLE = True
    print("✅ Speech recognition support available")
except ImportError:
    print("❌ Speech recognition not available")

try:
    from huggingface_hub import login
    from transformers import pipeline

    # Initialize the transformers-specific flags
    TRANSFORMERS_AVAILABLE = True

    # Try to set up sentiment analysis
    try:
        sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
        SENTIMENT_ANALYSIS_AVAILABLE = True
        print("✅ Sentiment analysis model loaded and ready")
    except Exception as e:
        print(f"❌ Sentiment analysis model couldn't be loaded: {str(e)}")

    print("✅ Hugging Face support available")
except ImportError:
    print("❌ Hugging Face libraries not available, LLM functionality will be limited")

# Try loading the datasets library for sentiment dataset
try:
    from datasets import load_dataset
    DATASETS_AVAILABLE = True
    print("✅ Hugging Face datasets library available")
except ImportError:
    print("❌ Hugging Face datasets library not available")

print("\nSection 1 complete: Setup and imports finished.\n")

✅ Machine learning support available
✅ TensorFlow/Keras support available for neural networks
✅ Text-to-speech support available
✅ Speech recognition support available


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


✅ Sentiment analysis model loaded and ready
✅ Hugging Face support available
✅ Hugging Face datasets library available

Section 1 complete: Setup and imports finished.



# Section 2: API SETUP

In [ ]:
# ============== SECTION 2: API SETUP ==============
# Set up the Hugging Face API

def setup_huggingface_api():
    """Get Hugging Face API token from Colab userdata, environment, or user input"""
    hf_token = None

    # First try to get from Colab userdata
    if USING_COLAB:
        try:
            hf_token = userdata.get('HUGGINGFACE_API_KEY')
            print("Retrieved Hugging Face API key from Colab secrets")
        except Exception as e:
            print(f"Could not retrieve API key from Colab secrets: {e}")

    # If not found in Colab, check environment
    if not hf_token:
        hf_token = os.environ.get("HUGGINGFACE_API_KEY")

    # If still not found, ask user
    if not hf_token:
        hf_token = getpass("Enter your Hugging Face API token: ")
        os.environ["HUGGINGFACE_API_KEY"] = hf_token

    # Log in to Hugging Face
    login(token=hf_token)
    print("Hugging Face API setup complete!")
    return hf_token

# Execute this section
try:
    api_token = setup_huggingface_api()
    print("Section 2 complete: API setup successful.\n")
except Exception as e:
    print(f"Error setting up API: {e}")
    print("Please ensure you have a valid Hugging Face API token.\n")


Enter your Hugging Face API token: ··········
Hugging Face API setup complete!
Section 2 complete: API setup successful.



# Section 3: LLM Configuration (Optional)

In [ ]:
# ============== SECTION 3: LANGUAGE MODEL SETUP ==============
# Configure the LLM using Hugging Face API

def generate_with_api(prompt, model_id="microsoft/phi-2"):
    """Generate text using the Hugging Face API"""
    API_URL = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {os.environ['HUGGINGFACE_API_KEY']}"}

    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 1024,
            "temperature": 0.7,
            "top_p": 0.9,
            "do_sample": True
        }
    }

    # Add retry logic for API calls
    max_retries = 3
    retry_delay = 2  # seconds

    for attempt in range(max_retries):
        try:
            response = requests.post(API_URL, headers=headers, json=payload, timeout=30)

            if response.status_code == 200:
                result = response.json()
                if isinstance(result, list) and len(result) > 0:
                    # Extract response text
                    generated_text = result[0].get("generated_text", "")
                    # Remove the prompt from the response
                    if generated_text.startswith(prompt):
                        generated_text = generated_text[len(prompt):]
                    return generated_text.strip()
                return str(result)
            elif response.status_code == 503:
                print(f"Model is loading. Attempt {attempt+1}/{max_retries}. Waiting {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
            else:
                error_msg = f"Error: {response.status_code} - {response.text}"
                print(error_msg)
                if attempt == max_retries - 1:
                    return error_msg
                time.sleep(retry_delay)
        except Exception as e:
            error_msg = f"Error calling API: {str(e)}"
            print(error_msg)
            if attempt == max_retries - 1:
                return error_msg
            time.sleep(retry_delay)

    return "Failed to get response after multiple attempts"

# Test the API with a simple prompt
try:
    test_prompt = "Hello, this is a test."
    response = generate_with_api(test_prompt)
    print(f"API Test Response: {response}")

    # Create a reusable function for generating responses
    generate_response = lambda prompt: generate_with_api(prompt)
    print("Section 3 complete: Language model setup successful.\n")
except Exception as e:
    print(f"Error testing API: {e}")
    print("Section 3 incomplete: Language model setup failed.\n")

Model is loading. Attempt 1/3. Waiting 2 seconds...
Model is loading. Attempt 2/3. Waiting 4 seconds...
Model is loading. Attempt 3/3. Waiting 8 seconds...
API Test Response: Failed to get response after multiple attempts
Section 3 complete: Language model setup successful.



# Section 4: TTS

In [ ]:
# ============== SECTION 4: TEXT-TO-SPEECH AND SPEECH RECOGNITION ==============
# Setup for bilingual text-to-speech and speech recognition

def text_to_speech(text, lang='en'):
    """Convert text to speech using gTTS in the specified language"""
    if not TTS_AVAILABLE:
        print(f"Text-to-speech not available. Text content: {text}")
        return None

    try:
        # Check for a valid language code
        valid_langs = {'en': 'English', 'ar': 'Arabic'}
        if lang not in valid_langs:
            print(f"Language '{lang}' not supported. Defaulting to English.")
            lang = 'en'

        print(f"Converting text to speech ({valid_langs[lang]})...")
        tts = gTTS(text=text, lang=lang, slow=False)
        audio_file = "temp_audio.mp3"
        tts.save(audio_file)

        # Play the audio file using the operating system's default player
        if os.name == 'posix':  # For Linux/Mac
            os.system(f"afplay {audio_file}" if os.uname().sysname == 'Darwin' else f"mpg123 {audio_file}")
        else:  # For Windows
            os.system(f"start {audio_file}")

        return True
    except Exception as e:
        print(f"Error generating speech: {e}")
        print(f"Text content: {text}")
        return None

def speech_to_text(lang='en'):
    """Convert speech to text using the specified language"""
    if not SPEECH_RECOGNITION_AVAILABLE:
        print("Speech recognition not available.")
        text = input("Please type your response instead: ")
        return text

    recognizer = sr.Recognizer()

    try:
        # Map language codes to recognition language
        lang_map = {'en': 'en-US', 'ar': 'ar-SA'}
        recognition_lang = lang_map.get(lang, 'en-US')

        with sr.Microphone() as source:
            print(f"Listening for {lang} speech... (speak now)")
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.listen(source, timeout=10)

        print("Processing speech...")
        text = recognizer.recognize_google(audio, language=recognition_lang)
        print(f"Recognized: {text}")
        return text
    except sr.RequestError:
        print("Could not request results from speech recognition service")
    except sr.UnknownValueError:
        print("Could not understand audio")
    except Exception as e:
        print(f"Error in speech recognition: {e}")

    # Fallback to text input
    text = input("Please type your response instead: ")
    return text

# Test TTS with both languages
if TTS_AVAILABLE:
    print("Testing Text-to-Speech...")
    text_to_speech("Hello, this is a test in English.", "en")
    text_to_speech("مرحبا، هذا اختبار باللغة العربية.", "ar")

# Note: We won't test speech recognition here as it requires user interaction
print("Section 4 complete: Text-to-speech and speech recognition setup finished.\n")

Testing Text-to-Speech...
Converting text to speech (English)...
Converting text to speech (Arabic)...
Section 4 complete: Text-to-speech and speech recognition setup finished.



# Section 5a: Generate Data

In [ ]:
"""# Section 5a: Generate Data"""

# ============== SECTION 5: DATASET LOADING AND PREPARATION ==============
# Load and process the Software Engineering interview datasets

import pandas as pd
import numpy as np
import os
import traceback
from datasets import load_dataset
import time

# Check ML libraries availability
try:
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    ML_AVAILABLE = True
except ImportError:
    ML_AVAILABLE = False
    print("Warning: Machine learning libraries not available.")

def load_software_engineer_datasets(use_huggingface=True, generate_if_missing=False, num_samples=100000):
    """Load Software Engineering interview datasets from Hugging Face or generate synthetic data

    This function:
    1. Attempts to load data from Hugging Face
    2. Falls back to locally processed data if available
    3. Generates synthetic data as a last resort

    Args:
        use_huggingface: Whether to try loading from Hugging Face first
        generate_if_missing: If True and datasets are missing, generate synthetic data
        num_samples: Number of samples to generate if needed

    Returns:
        DataFrame with processed Software Engineering interview data
    """
    if not ML_AVAILABLE:
        print("Machine learning libraries not available. Cannot load and process datasets.")
        return None

    # First try loading from Hugging Face if requested
    if use_huggingface:
        try:
            print("Attempting to load Software Engineering interview dataset from Hugging Face...")
            # Load from Hugging Face
            splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
            df = pd.read_parquet("hf://datasets/Vineeshsuiii/Software_Engineering_interview_datasets/" + splits["train"])

            print(f"Loaded {len(df)} records from Hugging Face dataset")

            # Generate synthetic scores for each record
            print("Generating synthetic scores with controlled noise for realistic training...")

            # Function to add controlled noise to scores
            def add_noise(score, noise_level=0.15):
                noise = np.random.normal(0, noise_level * score, size=len(score))
                return np.clip(score + noise, 0, 100)

            # Generate initial scores based on text features
            def calculate_base_score(row):
                # Extract text features that might indicate question complexity
                question = str(row.get('question', ''))
                answer = str(row.get('answer', ''))

                # Simple complexity indicators
                complexity_indicators = ['complex', 'difficult', 'advanced', 'explain', 'design', 'implement']
                basic_indicators = ['basic', 'simple', 'describe', 'what is', 'define']

                # Calculate base score
                base = 75  # Start with average score

                # Adjust based on text length and complexity
                word_count = len(question.split())
                base += min(10, word_count / 20)  # Longer questions might be more complex

                # Adjust for complexity indicators
                complex_count = sum(1 for ind in complexity_indicators if ind in question.lower())
                basic_count = sum(1 for ind in basic_indicators if ind in question.lower())

                base += complex_count * 3
                base -= basic_count * 2

                # Add some randomness
                base += np.random.normal(0, 5)

                return np.clip(base, 50, 95)

            # Calculate initial scores based on question content
            df['base_score'] = df.apply(calculate_base_score, axis=1)

            # Generate component scores with realistic correlations and noise
            df['technical'] = add_noise(df['base_score'] + np.random.normal(0, 8, size=len(df)), noise_level=0.2)
            df['problem_solving'] = add_noise(df['base_score'] + np.random.normal(0, 7, size=len(df)), noise_level=0.18)
            df['communication'] = add_noise(df['base_score'] + np.random.normal(-2, 10, size=len(df)), noise_level=0.25)
            df['cultural_fit'] = add_noise(df['base_score'] + np.random.normal(-5, 12, size=len(df)), noise_level=0.3)

            # Introduce some outliers (about 5% of the data)
            outlier_mask = np.random.choice([True, False], size=len(df), p=[0.05, 0.95])
            for col in ['technical', 'problem_solving', 'communication', 'cultural_fit']:
                df.loc[outlier_mask, col] = np.random.choice([
                    np.random.uniform(30, 50),  # Low outliers
                    np.random.uniform(90, 100)  # High outliers
                ], size=sum(outlier_mask))

            # Calculate overall score with weighted average and additional noise
            df['overall'] = add_noise(
                0.4 * df['technical'] +
                0.3 * df['problem_solving'] +
                0.2 * df['communication'] +
                0.1 * df['cultural_fit'],
                noise_level=0.1
            )

            # Round all scores to integers
            score_columns = ['technical', 'communication', 'problem_solving', 'cultural_fit', 'overall']
            for col in score_columns:
                df[col] = df[col].round().astype(int)
                df[col] = df[col].clip(0, 100)  # Ensure within valid range

            # Drop the temporary base_score column
            df = df.drop('base_score', axis=1)

            print("\nGenerated synthetic scores with realistic noise and correlations")
            print(f"Score statistics:")
            for col in score_columns:
                print(f"- {col}:")
                print(f"  Range: {df[col].min()}-{df[col].max()}")
                print(f"  Mean: {df[col].mean():.1f}")
                print(f"  Std: {df[col].std():.1f}")

            # Process and save the dataset
            ml_dataset = process_dataset_for_ml(df)
            ml_dataset.to_csv("processed_se_interview_data.csv", index=False)
            print(f"\nProcessed dataset saved with {len(ml_dataset)} records")
            return ml_dataset

        except Exception as e:
            print(f"Error loading from Hugging Face: {e}")
            print("Falling back to local processed dataset or synthetic data generation...")

    # Then try to load local processed dataset
    try:
        ml_dataset = pd.read_csv("processed_se_interview_data.csv")
        print(f"Loaded pre-processed Software Engineering dataset with {len(ml_dataset)} records")
        return ml_dataset
    except FileNotFoundError:
        print("No processed Software Engineering dataset found, checking raw data files...")

    # Try loading from raw files
    try:
        # Load the datasets
        questions_df = pd.read_csv("se_interview_questions.csv")
        evaluations_df = pd.read_csv("se_evaluation_scores.csv")

        print(f"Loaded datasets:")
        print(f"- SE Interview Questions: {len(questions_df)} records")
        print(f"- SE Evaluation Scores: {len(evaluations_df)} records")

        # Merge datasets
        merged_df = pd.merge(evaluations_df, questions_df, on='question_id', how='inner')
        print(f"- Merged Dataset: {len(merged_df)} records after joining")

        # Check if we have enough data
        if len(merged_df) < 1000:
            print(f"Warning: Only {len(merged_df)} records available for training.")
            if generate_if_missing:
                print(f"Generating {num_samples} synthetic records as requested...")
                from generate_dataset import generate_software_engineer_dataset
                return generate_software_engineer_dataset(num_samples)
            else:
                print("Consider generating more data for better model training.")

        # Process for ML using the helper function
        ml_dataset = process_dataset_for_ml(merged_df)
        ml_dataset.to_csv("processed_se_interview_data.csv", index=False)

        return ml_dataset

    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Required dataset files not found.")

        if generate_if_missing:
            print(f"Generating {num_samples} synthetic Software Engineer records...")
            from generate_dataset import generate_software_engineer_dataset
            return generate_software_engineer_dataset(num_samples)
        else:
            print("To generate synthetic data, call load_software_engineer_datasets(generate_if_missing=True)")
            return None
    except Exception as e:
        print(f"Error processing datasets: {e}")
        traceback.print_exc()
        return None

def process_dataset_for_ml(data):
    """Process a dataset to create ML-ready features for Software Engineering interviews.

    Handles both HuggingFace datasets and synthetic data formats.
    """
    print("Processing dataset for machine learning...")
    processed_df = data.copy()
    feature_columns = []

    # Ensure we have the expected target columns
    required_targets = ['technical', 'communication', 'problem_solving', 'cultural_fit', 'overall']
    missing_targets = [target for target in required_targets if target not in processed_df.columns]

    if missing_targets:
        print(f"Generating synthetic values for missing targets: {missing_targets}")

        # Generate base score if no score/overall exists
        if 'score' in processed_df.columns:
            base = processed_df['score']
        elif 'overall' in processed_df.columns:
            base = processed_df['overall']
        else:
            base = np.random.randint(60, 100, size=len(processed_df))

        # Generate missing targets with correlation to base score
        for target in missing_targets:
            processed_df[target] = np.clip(base + np.random.randint(-10, 11, size=len(processed_df)), 0, 100)
            print(f"Generated synthetic values for {target}")

    # Basic numeric features
    if 'question_length' not in processed_df.columns and 'question' in processed_df.columns:
        processed_df['question_length'] = processed_df['question'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
        feature_columns.append('question_length')
    else:
        if 'question_length' in processed_df.columns:
            feature_columns.append('question_length')

    if 'word_count' not in processed_df.columns and 'question' in processed_df.columns:
        processed_df['word_count'] = processed_df['question'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)
        feature_columns.append('word_count')
    else:
        if 'word_count' in processed_df.columns:
            feature_columns.append('word_count')

    # Answer-based features if available
    if 'answer' in processed_df.columns:
        if 'answer_length' not in processed_df.columns:
            processed_df['answer_length'] = processed_df['answer'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
        if 'answer_word_count' not in processed_df.columns:
            processed_df['answer_word_count'] = processed_df['answer'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)

        feature_columns.extend(['answer_length', 'answer_word_count'])

    # Handle candidate_id if present
    if 'candidate_id' in processed_df.columns:
        feature_columns.append('candidate_id')

    # Date features
    if 'evaluation_date' in processed_df.columns:
        processed_df['evaluation_date'] = pd.to_datetime(processed_df['evaluation_date'])
        processed_df['month'] = processed_df['evaluation_date'].dt.month
        processed_df['year'] = processed_df['evaluation_date'].dt.year
        feature_columns.extend(['month', 'year'])

    # Handle categorical features - create dummies
    categorical_features = []

    if 'question_type' in processed_df.columns:
        question_type_dummies = pd.get_dummies(processed_df['question_type'], prefix='question_type')
        categorical_features.append(question_type_dummies)

    if 'difficulty_level' in processed_df.columns:
        difficulty_dummies = pd.get_dummies(processed_df['difficulty_level'], prefix='difficulty')
        categorical_features.append(difficulty_dummies)

    if 'topic' in processed_df.columns:
        topic_dummies = pd.get_dummies(processed_df['topic'], prefix='topic')
        categorical_features.append(topic_dummies)

    # Create the ML dataset
    ml_df = processed_df[feature_columns].copy() if feature_columns else pd.DataFrame()

    # Add categorical features
    for cat_feature in categorical_features:
        ml_df = pd.concat([ml_df, cat_feature], axis=1)

    # Add target variables (including the synthetic ones we generated)
    for target in required_targets:
        if target in processed_df.columns:
            ml_df[target] = processed_df[target]

    # Create recommendation classification if overall score is available
    if 'overall' in ml_df.columns:
        ml_df['recommendation'] = pd.cut(
            ml_df['overall'],
            bins=[0, 60, 80, 101],
            labels=['Not Recommended', 'Consider', 'Strongly Recommended']
        )

    # Keep original text columns for reference
    text_columns = ['question', 'answer', 'question_type', 'difficulty_level', 'topic']
    for col in text_columns:
        if col in processed_df.columns:
            ml_df[col + '_text'] = processed_df[col]

    print(f"Dataset processed with {len(ml_df)} samples and {len(ml_df.columns)} features.")
    return ml_df

# Replace the previous data loading with our new Software Engineering focused approach
if ML_AVAILABLE:
    print("Loading and preprocessing Software Engineering interview datasets...")
    training_data = load_software_engineer_datasets(use_huggingface=True)
    if training_data is not None:
        print("\nTraining data sample:")
        print(training_data.head())
    else:
        print("Failed to load Software Engineering interview datasets. Please check for errors above.")
else:
    print("Skipping dataset loading due to missing ML libraries.")

print("Section 5 complete: Software Engineering dataset preparation finished.\n")

Loading and preprocessing Software Engineering interview datasets...
Attempting to load Software Engineering interview dataset from Hugging Face...
Loaded 2858 records from Hugging Face dataset
Generating synthetic scores with controlled noise for realistic training...

Generated synthetic scores with realistic noise and correlations
Score statistics:
- technical:
  Range: 20-100
  Mean: 74.0
  Std: 17.3
- communication:
  Range: 0-100
  Mean: 71.1
  Std: 19.8
- problem_solving:
  Range: 30-100
  Mean: 74.2
  Std: 16.0
- cultural_fit:
  Range: 0-100
  Mean: 67.4
  Std: 22.2
- overall:
  Range: 34-100
  Mean: 72.9
  Std: 12.2
Processing dataset for machine learning...
Dataset processed with 2858 samples and 6 features.

Processed dataset saved with 2858 records

Training data sample:
   technical  communication  problem_solving  cultural_fit  overall  \
0         67             94               71            20       73   
1         71             46               81            61      

# Section 5b: Data Retrieval

In [ ]:
"""# Section 5b: Data Retrieval"""

import pandas as pd
import numpy as np
import random
import os
import re
import string
from datetime import datetime, timedelta
from datasets import load_dataset

# Import our hybrid dataset generator
try:
    from hybrid_dataset_generator import generate_hybrid_dataset
    HYBRID_GENERATOR_AVAILABLE = True
except ImportError:
    HYBRID_GENERATOR_AVAILABLE = False
    print("Hybrid dataset generator not available, will use standard methods")

def load_hf_dataset():
    """Load Software Engineering interview dataset from Hugging Face"""
    print("Loading Software Engineering interview dataset from Hugging Face...")

    try:
        # Load the dataset from Hugging Face
        splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
        df = pd.read_parquet("hf://datasets/Vineeshsuiii/Software_Engineering_interview_datasets/" + splits["train"])

        # Filter for Software Engineer roles only
        if 'profession' in df.columns:
            df = df[df['profession'] == 'Software Engineer']
            print(f"Filtered dataset to include only Software Engineer roles: {len(df)} records")
        else:
            print("Warning: 'profession' column not found in dataset. Using all data.")

        # Ensure we have all required target columns
        required_targets = ['technical', 'communication', 'problem_solving', 'cultural_fit', 'overall']
        missing_targets = [target for target in required_targets if target not in df.columns]

        if missing_targets:
            print(f"Warning: Missing target columns: {missing_targets}")
            print("Will generate synthetic values for missing targets")

            # Generate missing targets with some correlation to existing ones
            for target in missing_targets:
                if 'score' in df.columns:  # Use overall score as base if available
                    base = df['score']
                elif 'overall' in df.columns and target != 'overall':
                    base = df['overall']
                else:
                    base = np.random.randint(60, 100, size=len(df))

                # Add some random variation
                df[target] = np.clip(base + np.random.randint(-10, 11, size=len(df)), 0, 100)

        # Create any missing features that might be needed for modeling
        if 'question_length' not in df.columns and 'question' in df.columns:
            df['question_length'] = df['question'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

        if 'word_count' not in df.columns and 'question' in df.columns:
            df['word_count'] = df['question'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)

        print(f"Dataset loaded successfully with {len(df)} records")
        return df

    except Exception as e:
        print(f"Error loading dataset from Hugging Face: {str(e)}")
        print("Falling back to synthetic data generation...")
        return None

def generate_software_engineer_dataset(num_samples=10000):
    """Generate a synthetic dataset focused only on Software Engineer roles"""
    print(f"Generating {num_samples} Software Engineer interview samples...")

    # Define data distributions for Software Engineer questions
    question_types = ["Technical", "Behavioral", "Problem-Solving", "Cultural Fit", "Leadership"]
    difficulty_levels = ["Easy", "Medium", "Hard"]

    # Software Engineer specific topics
    se_topics = [
        "Algorithms", "Data Structures", "Object-Oriented Programming",
        "System Design", "Database Design", "Web Development",
        "DevOps", "Cloud Infrastructure", "Testing", "Debugging",
        "Version Control", "API Design", "Microservices", "Security"
    ]

    # Ensure a more balanced distribution across categories
    # For a 100K dataset, aim for roughly 25K weak, 50K average, 25K strong
    weak_proportion = 0.25
    average_proportion = 0.5
    strong_proportion = 0.25

    # Calculate sample counts
    weak_count = int(num_samples * weak_proportion)
    average_count = int(num_samples * average_proportion)
    strong_count = num_samples - weak_count - average_count

    # Generate questions and evaluations
    questions_data = []
    evaluations_data = []

    # Track the current counts for each category
    current_weak = 0
    current_average = 0
    current_strong = 0

    # Generate short responses for some of the weak examples (minimal/poor answers)
    minimal_responses = [
        "I don't know.",
        "Not sure",
        "No idea",
        "idk",
        "Skip",
        "I have no experience with that",
        "Never done that before",
        "What do you mean?",
        "Can you explain the question?",
        "This isn't my area",
        "I'm not familiar with this",
        "I'd have to look that up",
        "I haven't worked on that",
        "I'll get back to you on that",
        "That's not in my skill set",
        "Haven't learned that yet",
        "I'm confused by your question",
        "Can we move to another question?",
        "I'm drawing a blank",
        "I'd need to ask someone else",
    ]

    for i in range(1, num_samples + 1):
        # Generate question
        question_id = i
        topic = random.choice(se_topics)
        question_type = random.choice(question_types)
        difficulty = random.choice(difficulty_levels)

        # Determine which category this sample should be (weak, average, strong)
        if current_weak < weak_count:
            target_category = "weak"
            current_weak += 1
        elif current_average < average_count:
            target_category = "average"
            current_average += 1
        else:
            target_category = "strong"
            current_strong += 1

        # Generate questions based on type
        if question_type == "Technical":
            question = f"Explain how you would implement {topic} in a real-world software project."
        elif question_type == "Behavioral":
            question = f"Describe a time when you had to collaborate with others to solve a {topic.lower()} challenge."
        elif question_type == "Problem-Solving":
            question = f"How would you approach debugging a complex issue related to {topic}?"
        elif question_type == "Cultural Fit":
            question = f"How do you stay up-to-date with the latest developments in {topic}?"
        else:  # Leadership
            question = f"Tell me about a time you led a project involving {topic}."

        # Add to questions data
        questions_data.append({
            "id": question_id,
            "question_text": question,
            "question_type": question_type,
            "topic": topic,
            "difficulty": difficulty
        })

        # Generate answer based on target category
        if target_category == "weak":
            # For weak responses, we'll have some minimal responses and some longer but low-quality ones
            if random.random() < 0.4:  # 40% chance of minimal response
                answer = random.choice(minimal_responses)
                answer_length = "Very Short"
                technical_accuracy = "Low"
                communication_clarity = "Low"
                problem_solving = "Weak"
                cultural_fit = "Poor"
            else:
                answer = f"I think {topic} is important but I'm not very familiar with it. I would probably need to do some research to understand how to apply it properly."
                answer_length = "Short"
                technical_accuracy = random.choices(["Low", "Medium"], weights=[0.8, 0.2])[0]
                communication_clarity = random.choices(["Low", "Medium"], weights=[0.7, 0.3])[0]
                problem_solving = random.choices(["Weak", "Adequate"], weights=[0.8, 0.2])[0]
                cultural_fit = random.choices(["Poor", "Neutral"], weights=[0.7, 0.3])[0]

            # Set scores in the appropriate range for weak responses
            technical_score = random.uniform(10, 35)
            communication_score = random.uniform(10, 35)
            problem_solving_score = random.uniform(10, 35)
            cultural_fit_score = random.uniform(10, 35)
            overall_score = (technical_score + communication_score + problem_solving_score + cultural_fit_score) / 4

        elif target_category == "average":
            answer = f"When working with {topic}, I try to follow best practices. I've used it in a few projects before. " \
                     f"For example, in one project, we implemented a {topic} solution that helped improve our process."

            answer_length = random.choices(["Medium", "Long"], weights=[0.7, 0.3])[0]
            technical_accuracy = random.choices(["Medium", "High"], weights=[0.8, 0.2])[0]
            communication_clarity = random.choices(["Medium", "High"], weights=[0.7, 0.3])[0]
            problem_solving = random.choices(["Adequate", "Strong"], weights=[0.7, 0.3])[0]
            cultural_fit = random.choices(["Neutral", "Good"], weights=[0.6, 0.4])[0]

            # Set scores in the appropriate range for average responses
            technical_score = random.uniform(40, 70)
            communication_score = random.uniform(40, 70)
            problem_solving_score = random.uniform(40, 70)
            cultural_fit_score = random.uniform(40, 70)
            overall_score = (technical_score + communication_score + problem_solving_score + cultural_fit_score) / 4

        else:  # strong
            answer = f"I have extensive experience with {topic}. In my previous role, I led the implementation of a " \
                     f"{topic} solution that resulted in a 30% improvement in performance. " \
                     f"I follow a systematic approach where I first analyze requirements, then design a solution " \
                     f"considering scalability and maintainability. I've also mentored junior engineers on best practices for {topic}."

            answer_length = "Long"
            technical_accuracy = "High"
            communication_clarity = random.choices(["Medium", "High"], weights=[0.2, 0.8])[0]
            problem_solving = "Strong"
            cultural_fit = random.choices(["Neutral", "Good"], weights=[0.1, 0.9])[0]

            # Set scores in the appropriate range for strong responses
            technical_score = random.uniform(75, 100)
            communication_score = random.uniform(75, 100)
            problem_solving_score = random.uniform(75, 100)
            cultural_fit_score = random.uniform(75, 100)
            overall_score = (technical_score + communication_score + problem_solving_score + cultural_fit_score) / 4

        # Add to evaluations data
        evaluations_data.append({
            "id": question_id,
            "answer_text": answer,
            "technical_score": technical_score,
            "communication_score": communication_score,
            "problem_solving_score": problem_solving_score,
            "cultural_fit_score": cultural_fit_score,
            "overall_score": overall_score,
            "answer_length": answer_length,
            "technical_accuracy": technical_accuracy,
            "communication_clarity": communication_clarity,
            "problem_solving": problem_solving,
            "cultural_fit": cultural_fit,
            "category": target_category.capitalize(),  # "Weak", "Average", or "Strong"
            "evaluation_date": datetime.now().strftime("%Y-%m-%d")  # Add today's date
        })

        # Show progress
        if i % 2000 == 0 or i == num_samples:
            print(f"Generated {i}/{num_samples} questions and {len(evaluations_data)} evaluations")

    # Create DataFrames
    questions_df = pd.DataFrame(questions_data)
    evaluations_df = pd.DataFrame(evaluations_data)

    # Merge the datasets
    merged_df = pd.merge(questions_df, evaluations_df, on="id")

    # Save to CSV
    print("Saving CSV files...")
    merged_df.to_csv("se_interview_questions.csv", index=False)

    print(f"Dataset generation complete!")
    print(f"- Questions dataset: {len(questions_df)} records")
    print(f"- Evaluations dataset: {len(evaluations_df)} records")

    # Now create a merged dataset for ML
    print("Creating merged ML dataset...")
    merged_df = pd.merge(evaluations_df, questions_df, on='id', how='inner')

    # Create dummy variables for categorical features
    question_type_dummies = pd.get_dummies(merged_df['question_type'], prefix='question_type')
    difficulty_dummies = pd.get_dummies(merged_df['difficulty'], prefix='difficulty')
    topic_dummies = pd.get_dummies(merged_df['topic'], prefix='topic')

    # Create date-related features
    try:
        merged_df['evaluation_date'] = pd.to_datetime(merged_df['evaluation_date'])
        merged_df['month'] = merged_df['evaluation_date'].dt.month
        merged_df['year'] = merged_df['evaluation_date'].dt.year
    except (KeyError, ValueError) as e:
        print(f"Warning: Error processing dates: {e}. Using default values.")
        # Use current date information if there's an issue
        current_date = datetime.now()
        merged_df['month'] = current_date.month
        merged_df['year'] = current_date.year

    # Text-based features from questions
    merged_df['question_length'] = merged_df['question_text'].apply(len)
    merged_df['word_count'] = merged_df['question_text'].apply(lambda x: len(str(x).split()))

    # Create the final feature dataset
    features = pd.concat([
        merged_df[['id', 'question_length', 'word_count', 'month', 'year']],
        question_type_dummies,
        difficulty_dummies,
        topic_dummies
    ], axis=1)

    # Create ML dataset
    ml_dataset = features.copy()
    ml_dataset['score'] = merged_df['overall_score']
    ml_dataset['overall'] = merged_df['overall_score']
    ml_dataset['technical'] = merged_df['technical_score']
    ml_dataset['communication'] = merged_df['communication_score']
    ml_dataset['problem_solving'] = merged_df['problem_solving_score']
    ml_dataset['cultural_fit'] = merged_df['cultural_fit_score']

    # Map recommendation categories
    def map_to_category(score):
        if score >= 75:
            return 2  # Strong
        elif score >= 40:
            return 1  # Average
        else:
            return 0  # Weak

    # Add classifier target that maps directly to the category field (Weak=0, Average=1, Strong=2)
    ml_dataset['classifier'] = merged_df['category'].map({'Weak': 0, 'Average': 1, 'Strong': 2})

    # Also calculate the classifier based on overall score (use this as a fallback)
    ml_dataset['classifier_score_based'] = ml_dataset['overall'].apply(map_to_category)

    # Store text fields for reference
    ml_dataset['question_text'] = merged_df['question_text']
    ml_dataset['question_type'] = merged_df['question_type']
    ml_dataset['difficulty_level'] = merged_df['difficulty']
    ml_dataset['topic'] = merged_df['topic']

    # Add the answer_text column that's required by process_dataset_for_ml
    if 'answer_text' not in ml_dataset.columns:
        # Create sample answers based on the category
        def generate_sample_answer(row):
            quality = "excellent" if row['classifier'] == 2 else "average" if row['classifier'] == 1 else "poor"
            question = row['question_text']
            topic = row['topic']

            if quality == "excellent":
                return f"This is a comprehensive answer to the {topic} question about '{question[:30]}...'. The answer demonstrates strong technical knowledge and detailed understanding."
            elif quality == "average":
                return f"This is a satisfactory answer to the {topic} question about '{question[:30]}...'. The answer shows basic understanding but lacks some details."
            else:
                return f"This is a basic answer to the {topic} question about '{question[:30]}...'. The answer has several gaps in understanding."

        ml_dataset['answer_text'] = ml_dataset.apply(generate_sample_answer, axis=1)
        print("Generated answer_text column required for processing")

    # Save processed dataset
    ml_dataset.to_csv("processed_se_interview_data.csv", index=False)
    print(f"Processed dataset saved with {len(ml_dataset)} records")

    return ml_dataset

def get_interview_dataset(num_samples=10000, use_huggingface=True, evaluation_method='hybrid'):
    """Main function to get software engineering interview dataset

    Args:
        num_samples: Number of samples to generate if needed
        use_huggingface: Whether to try using Hugging Face data
        evaluation_method: 'direct' to use scores from HF, 'hybrid' to generate scores from answer quality

    Returns:
        Processed dataset ready for ML training
    """

    # Try hybrid approach with answer evaluation
    if use_huggingface and evaluation_method == 'hybrid' and HYBRID_GENERATOR_AVAILABLE:
        print("Using hybrid approach with Hugging Face Q&A and generated evaluations...")
        hybrid_data = generate_hybrid_dataset(num_samples)
        if hybrid_data is not None:
            return hybrid_data

    # Direct Hugging Face approach
    if use_huggingface and evaluation_method == 'direct':
        hf_data = load_hf_dataset()
        if hf_data is not None:
            return hf_data

    # Fall back to synthetic data generation
    print(f"Generating synthetic interview dataset with {num_samples} samples...")
    raw_data = generate_software_engineer_dataset(num_samples)

    # Process the raw data into ML-ready format
    processed_data = process_dataset_for_ml(raw_data)

    if processed_data is not None:
        print(f"Successfully processed {len(processed_data)} records")
        return processed_data
    else:
        print("Warning: Data processing failed, returning raw data")
        # Check if raw_data already has essential ML columns
        if all(col in raw_data.columns for col in ['classifier', 'technical', 'communication', 'problem_solving', 'overall']):
            print("Raw data already contains essential ML columns, using as is")
        else:
            # Add minimal processing to ensure raw_data is usable
            print("Adding minimal processing to raw data")
            if 'classifier' not in raw_data.columns and 'overall' in raw_data.columns:
                raw_data['classifier'] = raw_data['overall'].apply(map_to_category)
        return raw_data

def load_sentiment_dataset():
    """Load multiclass sentiment analysis dataset from Hugging Face"""
    print("Loading sentiment analysis dataset from Hugging Face...")

    try:
        # Load the sentiment analysis dataset from Hugging Face
        sentiment_dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

        # Convert to DataFrame for easier processing
        df = pd.DataFrame({
            'text': sentiment_dataset['train']['text'],
            'sentiment_label': sentiment_dataset['train']['label'],
            'sentiment': sentiment_dataset['train']['sentiment']
        })

        print(f"Loaded {len(df)} records from sentiment analysis dataset")
        return df
    except Exception as e:
        print(f"Error loading sentiment dataset: {e}")
        return None

def process_dataset_for_ml(data):
    """
    Process raw data into a format suitable for ML modeling.
    This extracts features from interview responses and prepares the data for training.

    Args:
        data: Raw dataset with interview questions and responses

    Returns:
        Processed dataset with features and targets
    """
    if 'answer_text' not in data.columns:
        print("Warning: answer_text column not found.")

        # Try to create an answer_text column from existing data
        if 'response' in data.columns:
            print("Using 'response' column as answer_text")
            data['answer_text'] = data['response']
        elif 'answer' in data.columns:
            print("Using 'answer' column as answer_text")
            data['answer_text'] = data['answer']
        elif 'question_text' in data.columns and 'classifier' in data.columns:
            print("Generating sample answer_text from question_text and classifier")
            # Generate simple dummy answers based on the classification
            def generate_fallback_answer(row):
                quality = "excellent" if row['classifier'] == 2 else "average" if row['classifier'] == 1 else "poor"
                question = row['question_text'] if isinstance(row['question_text'], str) else "the question"
                question_prefix = question[:30] if len(question) > 30 else question

                if quality == "excellent":
                    return f"This is a detailed answer to {question_prefix}... demonstrating excellent knowledge."
                elif quality == "average":
                    return f"This is an adequate answer to {question_prefix}... with some correct information."
                else:
                    return f"This is a minimal answer to {question_prefix}... with several inaccuracies."

            data['answer_text'] = data.apply(generate_fallback_answer, axis=1)
        else:
            print("Cannot generate answer_text. Cannot process this dataset.")
            return None

    print("Processing dataset for ML...")

    try:
        # Try to import our feature extraction function
        import sys
        from pathlib import Path
        import os

        # Get current directory in a way that works in both scripts and notebooks
        try:
            # For regular Python scripts
            current_dir = Path(__file__).parent
        except NameError:
            # For Jupyter/Colab notebooks
            import os
            current_dir = Path(os.getcwd())

        parent_dir = current_dir.parent
        if str(parent_dir) not in sys.path:
            sys.path.append(str(parent_dir))

        # Import feature extraction
        try:
            from section_8_interview_process import extract_features_from_text
            FEATURE_EXTRACTOR_AVAILABLE = True
        except ImportError:
            FEATURE_EXTRACTOR_AVAILABLE = False
            print("Feature extractor not available, will use basic features")

        # Process each row to extract features
        processed_rows = []

        for idx, row in data.iterrows():
            if idx % 1000 == 0:
                print(f"Processing row {idx}/{len(data)}...")

            # Get the answer text
            answer = row['answer_text']

            # Extract features
            if FEATURE_EXTRACTOR_AVAILABLE:
                features = extract_features_from_text(answer)
            else:
                # Basic feature extraction
                word_count = len(answer.split())
                char_count = len(answer)

                # Detect minimal responses
                minimal_response = word_count <= 5
                minimal_response_penalty = 0.2 if minimal_response else 1.0

                # Calculate basic metrics
                avg_word_length = char_count / max(word_count, 1)
                sentence_count = answer.count('.') + answer.count('!') + answer.count('?')

                # Simple word-based sentiment approximation (very basic)
                positive_words = ["good", "great", "excellent", "amazing", "awesome", "yes"]
                negative_words = ["bad", "poor", "no", "don't", "cant", "idk", "not"]

                positive_count = sum(1 for word in positive_words if word in answer.lower())
                negative_count = sum(1 for word in negative_words if word in answer.lower())
                sentiment = 0.3 if minimal_response else (0.5 + (positive_count - negative_count) * 0.1)

                # Create feature dictionary
                features = {
                    'feature_0': float(word_count * minimal_response_penalty),  # Word count
                    'feature_1': float(char_count * minimal_response_penalty),  # Character count
                    'feature_2': float(avg_word_length),                        # Avg word length
                    'feature_3': float(sentence_count * minimal_response_penalty),  # Sentence count
                    'feature_4': float(0 if minimal_response else 2),           # Technical keywords
                    'feature_5': float(0 if minimal_response else 2),           # Communication keywords
                    'feature_6': float(0 if minimal_response else 2),           # Problem-solving keywords
                    'feature_7': float(0 if minimal_response else 2),           # Cultural fit keywords
                    'feature_8': float(30 if minimal_response else 60),         # Confidence score
                    'feature_9': float(0.1 if minimal_response else 0.5),       # Keyword ratio
                    'feature_10': float(sentiment)                              # Sentiment
                }

            # Add targets
            if 'technical_score' in row:
                features['technical'] = row['technical_score']
            if 'communication_score' in row:
                features['communication'] = row['communication_score']
            if 'problem_solving_score' in row:
                features['problem_solving'] = row['problem_solving_score']
            if 'cultural_fit_score' in row:
                features['cultural_fit'] = row['cultural_fit_score']
            if 'overall_score' in row:
                features['overall'] = row['overall_score']
            if 'category' in row:
                category_map = {'Weak': 0, 'Average': 1, 'Strong': 2}
                features['classifier'] = category_map.get(row['category'], 1)  # Default to Average if missing

            processed_rows.append(features)

        # Convert to DataFrame
        processed_df = pd.DataFrame(processed_rows)
        print(f"Processed {len(processed_df)} rows successfully")

        # Ensure we have all the required feature columns
        required_features = [f'feature_{i}' for i in range(11)]
        for feature in required_features:
            if feature not in processed_df.columns:
                processed_df[feature] = 0

        # Ensure we have all the required target columns
        required_targets = ['technical', 'communication', 'problem_solving', 'cultural_fit', 'overall', 'classifier']
        for target in required_targets:
            if target not in processed_df.columns:
                processed_df[target] = 50  # Default value (except for classifier)

        if 'classifier' not in processed_df.columns:
            processed_df['classifier'] = 1  # Default to Average

        return processed_df

    except Exception as e:
        print(f"Error processing dataset: {e}")
        import traceback
        traceback.print_exc()
        return None

    return None

if __name__ == "__main__":
    # Generate hybrid dataset with answer evaluations by default
    get_interview_dataset(num_samples=100000, use_huggingface=True, evaluation_method='hybrid')

Hybrid dataset generator not available, will use standard methods
Generating synthetic interview dataset with 100000 samples...
Generating 100000 Software Engineer interview samples...
Generated 2000/100000 questions and 2000 evaluations
Generated 4000/100000 questions and 4000 evaluations
Generated 6000/100000 questions and 6000 evaluations
Generated 8000/100000 questions and 8000 evaluations
Generated 10000/100000 questions and 10000 evaluations
Generated 12000/100000 questions and 12000 evaluations
Generated 14000/100000 questions and 14000 evaluations
Generated 16000/100000 questions and 16000 evaluations
Generated 18000/100000 questions and 18000 evaluations
Generated 20000/100000 questions and 20000 evaluations
Generated 22000/100000 questions and 22000 evaluations
Generated 24000/100000 questions and 24000 evaluations
Generated 26000/100000 questions and 26000 evaluations
Generated 28000/100000 questions and 28000 evaluations
Generated 30000/100000 questions and 30000 evaluation

# Section 7: Model Training

In [ ]:
"""# Section 7: Model Training"""

# ==== SECTION 7: TRAINING ML MODELS ====
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Initialize GPU availability flag
use_gpu = False
GPU_AVAILABLE = False
TORCH_GPU_AVAILABLE = False

# Check for GPU availability
try:
    import tensorflow as tf
    print(f"TensorFlow version: {tf.__version__}")
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print(f"GPU devices available: {len(gpus)}")
        print(f"GPU details: {gpus}")
        # Set memory growth to avoid taking all GPU memory
        for gpu in gpus:
            try:
                tf.config.experimental.set_memory_growth(gpu, True)
            except Exception as e:
                print(f"Error setting memory growth: {e}")
        GPU_AVAILABLE = True
        use_gpu = True  # Set the global use_gpu flag
    else:
        print("No GPU devices detected by TensorFlow")
        GPU_AVAILABLE = False

    # Also check if CUDA is available for PyTorch
    try:
        import torch
        TORCH_GPU_AVAILABLE = torch.cuda.is_available()
        if TORCH_GPU_AVAILABLE:
            print(f"PyTorch CUDA available: {TORCH_GPU_AVAILABLE}")
            print(f"CUDA device count: {torch.cuda.device_count()}")
            print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
            use_gpu = True  # Set the global use_gpu flag
        else:
            print("PyTorch CUDA not available")
    except Exception as e:
        print(f"Error checking PyTorch CUDA: {e}")
        TORCH_GPU_AVAILABLE = False

except Exception as e:
    print(f"Error checking TensorFlow GPU: {e}")
    GPU_AVAILABLE = False
    TORCH_GPU_AVAILABLE = False
    use_gpu = False

# Try to import Optuna for hyperparameter optimization
try:
    import optuna
    optuna_available = True
    print("Optuna available for hyperparameter optimization")
except ImportError:
    optuna_available = False
    print("Optuna not available, will use RandomizedSearchCV instead")

# Try to import XGBoost for advanced modeling
try:
    import xgboost as xgb
    xgboost_available = True
    print("XGBoost available for advanced modeling")
except ImportError:
    xgboost_available = False
    print("XGBoost not available, will use RandomForest instead")

# ==== 6.1: TRAINING REGRESSION MODELS ====

def train_regression_models(X_train, X_test, y_train, y_test, target_name, use_gpu=False, verbose=True):
    """
    Train and evaluate multiple regression models for a specific target

    Args:
        X_train: Training features
        X_test: Testing features
        y_train: Training target values
        y_test: Testing target values
        target_name: Name of the target variable
        use_gpu: Whether to use GPU acceleration
        verbose: Whether to print progress info

    Returns:
        Best performing model and its metrics
    """
    models = {}
    results = {}

    if verbose:
        print(f"\nTraining regression models for {target_name}...")
        print(f"GPU acceleration: {'Enabled' if use_gpu and (GPU_AVAILABLE or TORCH_GPU_AVAILABLE) else 'Disabled'}")

    # 1. Random Forest Regressor
    if verbose:
        print("1. Training Random Forest Regressor...")
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_mse = mean_squared_error(y_test, rf_pred)
    rf_mae = mean_absolute_error(y_test, rf_pred)
    rf_r2 = r2_score(y_test, rf_pred)

    models["random_forest"] = rf
    results["random_forest"] = {
        "mse": rf_mse,
        "mae": rf_mae,
        "r2": rf_r2
    }

    if verbose:
        print(f"   MSE: {rf_mse:.4f}, MAE: {rf_mae:.4f}, R²: {rf_r2:.4f}")

    # 2. Gradient Boosting Regressor
    if verbose:
        print("2. Training Gradient Boosting Regressor...")
    gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gb.fit(X_train, y_train)
    gb_pred = gb.predict(X_test)
    gb_mse = mean_squared_error(y_test, gb_pred)
    gb_mae = mean_absolute_error(y_test, gb_pred)
    gb_r2 = r2_score(y_test, gb_pred)

    models["gradient_boosting"] = gb
    results["gradient_boosting"] = {
        "mse": gb_mse,
        "mae": gb_mae,
        "r2": gb_r2
    }

    if verbose:
        print(f"   MSE: {gb_mse:.4f}, MAE: {gb_mae:.4f}, R²: {gb_r2:.4f}")

    # 3. XGBoost Regressor (if available)
    if xgboost_available:
        if verbose:
            print("3. Training XGBoost Regressor...")

        # Set up XGBoost parameters with GPU support if available
        if use_gpu and (GPU_AVAILABLE or TORCH_GPU_AVAILABLE):
            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'tree_method': 'gpu_hist',  # Use GPU acceleration
                'gpu_id': 0,
                'predictor': 'gpu_predictor',
                'learning_rate': 0.1,
                'max_depth': 6,
                'min_child_weight': 1,
                'n_estimators': 100,
                'early_stopping_rounds': 10,
                'verbosity': 1 if verbose else 0
            }
            if verbose:
                print("   Using GPU acceleration for XGBoost")
        else:
            params = {
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'tree_method': 'hist',  # CPU-based histogram algorithm
                'learning_rate': 0.1,
                'max_depth': 6,
                'min_child_weight': 1,
                'n_estimators': 100,
                'early_stopping_rounds': 10,
                'verbosity': 1 if verbose else 0
            }

        # Create DMatrix for efficient training
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        # Train model
        xgb_model = xgb.train(
            params,
            dtrain,
            num_boost_round=100,
            evals=[(dtest, 'test')],
            early_stopping_rounds=10,
            verbose_eval=verbose
        )

        # Make predictions
        xgb_pred = xgb_model.predict(dtest)
        xgb_mse = mean_squared_error(y_test, xgb_pred)
        xgb_mae = mean_absolute_error(y_test, xgb_pred)
        xgb_r2 = r2_score(y_test, xgb_pred)

        models["xgboost"] = xgb_model
        results["xgboost"] = {
            "mse": xgb_mse,
            "mae": xgb_mae,
            "r2": xgb_r2
        }

        if verbose:
            print(f"   MSE: {xgb_mse:.4f}, MAE: {xgb_mae:.4f}, R²: {xgb_r2:.4f}")

    # Find the best model based on MSE
    best_model_name = min(results, key=lambda x: results[x]['mse'])
    best_model = models[best_model_name]
    best_metrics = results[best_model_name]

    if verbose:
        print(f"\nBest model for {target_name}: {best_model_name}")
        print(f"  MSE: {best_metrics['mse']:.4f}")
        print(f"  MAE: {best_metrics['mae']:.4f}")
        print(f"  R²: {best_metrics['r2']:.4f}")

    return best_model, best_metrics

def optimize_hyperparameters(X_train, y_train, model_type='xgboost', use_gpu=False,
                            is_classification=False, n_trials=50, verbose=True):
    """
    Optimize hyperparameters for a given model type

    Args:
        X_train: Training features
        y_train: Training target values
        model_type: Type of model to optimize ('xgboost', 'random_forest', etc.)
        use_gpu: Whether to use GPU acceleration
        is_classification: Whether this is a classification task
        n_trials: Number of optimization trials
        verbose: Whether to print progress

    Returns:
        Optimized model
    """
    if optuna_available:
        if verbose:
            print(f"Optimizing {model_type} hyperparameters with Optuna...")

        def objective(trial):
            if model_type == 'xgboost' and xgboost_available:
                params = {
                    'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                    'max_depth': trial.suggest_int('max_depth', 3, 10),
                    'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                    'random_state': 42
                }

                if use_gpu:
                    params['tree_method'] = 'gpu_hist'
                    params['gpu_id'] = 0

                if is_classification:
                    model = xgb.XGBClassifier(**params)
                else:
                    model = xgb.XGBRegressor(**params)

            elif model_type == 'random_forest':
                params = {
                    'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                    'max_depth': trial.suggest_int('max_depth', 5, 20),
                    'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
                    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
                    'random_state': 42
                }

                if is_classification:
                    model = RandomForestClassifier(**params)
                else:
                    model = RandomForestRegressor(**params)

            else:
                # Default to RandomForest if model_type is not supported
                params = {
                    'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                    'max_depth': trial.suggest_int('max_depth', 5, 20),
                    'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
                    'random_state': 42
                }

                if is_classification:
                    model = RandomForestClassifier(**params)
                else:
                    model = RandomForestRegressor(**params)

            # Train and evaluate with cross-validation
            from sklearn.model_selection import cross_val_score

            if is_classification:
                score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
            else:
                # Use negative MSE for regression (Optuna minimizes the objective)
                score = -cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

            return score

        # Create and run the study
        if is_classification:
            direction = 'maximize'  # Maximize accuracy for classification
        else:
            direction = 'minimize'  # Minimize MSE for regression

        study = optuna.create_study(direction=direction)
        study.optimize(objective, n_trials=n_trials)

        # Get the best parameters
        best_params = study.best_params

        if verbose:
            print(f"Best parameters: {best_params}")

        # Create and return the best model
        if model_type == 'xgboost' and xgboost_available:
            if use_gpu:
                best_params['tree_method'] = 'gpu_hist'
                best_params['gpu_id'] = 0

            if is_classification:
                best_model = xgb.XGBClassifier(**best_params)
            else:
                best_model = xgb.XGBRegressor(**best_params)

        elif model_type == 'random_forest':
            if is_classification:
                best_model = RandomForestClassifier(**best_params)
            else:
                best_model = RandomForestRegressor(**best_params)

        else:
            if is_classification:
                best_model = RandomForestClassifier(**best_params)
            else:
                best_model = RandomForestRegressor(**best_params)

        # Train the final model
        best_model.fit(X_train, y_train)
        return best_model

    else:
        # Use RandomizedSearchCV if Optuna is not available
        if verbose:
            print(f"Optimizing {model_type} hyperparameters with RandomizedSearchCV...")

        if model_type == 'xgboost' and xgboost_available:
            param_grid = {
                'n_estimators': [50, 100, 200, 300],
                'learning_rate': [0.01, 0.05, 0.1, 0.2],
                'max_depth': [3, 5, 7, 9],
                'subsample': [0.6, 0.8, 1.0],
                'colsample_bytree': [0.6, 0.8, 1.0]
            }

            if is_classification:
                model = xgb.XGBClassifier(random_state=42)
            else:
                model = xgb.XGBRegressor(random_state=42)

            if use_gpu:
                model.set_params(tree_method='gpu_hist', gpu_id=0)

        elif model_type == 'random_forest':
            param_grid = {
                'n_estimators': [50, 100, 200, 300],
                'max_depth': [5, 10, 15, 20],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]
            }

            if is_classification:
                model = RandomForestClassifier(random_state=42)
            else:
                model = RandomForestRegressor(random_state=42)

        else:
            # Default to RandomForest
            param_grid = {
                'n_estimators': [50, 100, 200, 300],
                'max_depth': [5, 10, 15, 20],
                'min_samples_split': [2, 5, 10]
            }

            if is_classification:
                model = RandomForestClassifier(random_state=42)
            else:
                model = RandomForestRegressor(random_state=42)

        # Create and run the search
        if is_classification:
            scoring = 'accuracy'
        else:
            scoring = 'neg_mean_squared_error'

        search = RandomizedSearchCV(
            model, param_grid, n_iter=10, cv=5,
            scoring=scoring, random_state=42, n_jobs=-1
        )

        search.fit(X_train, y_train)

        if verbose:
            print(f"Best parameters: {search.best_params_}")

        return search.best_estimator_

# ==== 6.2: TRAINING CLASSIFICATION MODELS ====

def train_classifier(X_train, X_test, y_train, y_test, use_gpu=False, verbose=True):
    """
    Train and evaluate a classifier for the Strong/Average/Weak categories

    Args:
        X_train: Training features
        X_test: Testing features
        y_train: Training target values (0: Weak, 1: Average, 2: Strong)
        y_test: Testing target values
        use_gpu: Whether to use GPU acceleration
        verbose: Whether to print progress

    Returns:
        Best performing classifier model
    """
    if verbose:
        print("\nTraining classifier for Strong/Average/Weak categories...")
        print(f"GPU acceleration: {'Enabled' if use_gpu and (GPU_AVAILABLE or TORCH_GPU_AVAILABLE) else 'Disabled'}")

    models = {}
    results = {}

    # 1. Random Forest Classifier
    if verbose:
        print("1. Training Random Forest Classifier...")
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf_classifier.fit(X_train, y_train)
    rf_preds = rf_classifier.predict(X_test)
    rf_acc = accuracy_score(y_test, rf_preds)

    models["random_forest"] = rf_classifier
    results["random_forest"] = {
        "accuracy": rf_acc
    }

    if verbose:
        print(f"   Accuracy: {rf_acc:.4f}")

    # 2. XGBoost Classifier (if available)
    if xgboost_available:
        if verbose:
            print("2. Training XGBoost Classifier...")

        # Set up XGBoost parameters with GPU support if available
        if use_gpu and (GPU_AVAILABLE or TORCH_GPU_AVAILABLE):
            params = {
                'objective': 'multi:softmax',
                'eval_metric': 'mlogloss',
                'num_class': 3,  # 0: Weak, 1: Average, 2: Strong
                'tree_method': 'gpu_hist',  # Use GPU acceleration
                'gpu_id': 0,
                'predictor': 'gpu_predictor',
                'learning_rate': 0.1,
                'max_depth': 6,
                'min_child_weight': 1,
                'verbosity': 1 if verbose else 0
            }
            if verbose:
                print("   Using GPU acceleration for XGBoost")
        else:
            params = {
                'objective': 'multi:softmax',
                'eval_metric': 'mlogloss',
                'num_class': 3,  # 0: Weak, 1: Average, 2: Strong
                'tree_method': 'hist',  # CPU-based histogram algorithm
                'learning_rate': 0.1,
                'max_depth': 6,
                'min_child_weight': 1,
                'verbosity': 1 if verbose else 0
            }

        # Create DMatrix for efficient training
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        # Train model
        xgb_model = xgb.train(
            params,
            dtrain,
            num_boost_round=100,
            evals=[(dtest, 'test')],
            early_stopping_rounds=10,
            verbose_eval=verbose
        )

        # Make predictions
        xgb_preds = xgb_model.predict(dtest)
        xgb_preds = xgb_preds.astype(int)  # Convert to integers
        xgb_acc = accuracy_score(y_test, xgb_preds)

        models["xgboost"] = xgb_model
        results["xgboost"] = {
            "accuracy": xgb_acc
        }

        if verbose:
            print(f"   Accuracy: {xgb_acc:.4f}")

    # Find the best model based on accuracy
    best_model_name = max(results, key=lambda x: results[x]['accuracy'])
    best_model = models[best_model_name]

    if verbose:
        print(f"\nBest classifier: {best_model_name} (Accuracy: {results[best_model_name]['accuracy']:.4f})")
        print("\nClassification Report:")
        if best_model_name == "xgboost":
            best_preds = best_model.predict(dtest).astype(int)
        else:
            best_preds = best_model.predict(X_test)
        print(classification_report(y_test, best_preds,
                                  target_names=['Weak', 'Average', 'Strong']))

    return best_model

def generate_minimal_sample_data(num_samples=100000):
    """
    Generate a very minimal dataset for emergency fallback purposes.

    Args:
        num_samples: Number of samples to generate

    Returns:
        DataFrame with basic feature and target columns
    """
    print(f"Generating minimal emergency dataset with {num_samples} samples...")

    # Import needed libraries
    import numpy as np
    import pandas as pd

    # Create empty dataframe
    data = pd.DataFrame()

    # Generate features
    for i in range(11):
        feature_name = f'feature_{i}'
        # Random values calibrated to reasonable ranges
        data[feature_name] = np.random.rand(num_samples) * (100 if i in [0, 1, 8] else 10)

    # Generate target variables - ensure good distribution
    categories = np.random.choice([0, 1, 2], size=num_samples, p=[0.25, 0.5, 0.25])

    # Add scores based on categories
    data['technical'] = np.where(categories == 0, np.random.uniform(10, 35, num_samples),
                       np.where(categories == 1, np.random.uniform(40, 70, num_samples),
                                np.random.uniform(75, 95, num_samples)))

    data['communication'] = np.where(categories == 0, np.random.uniform(10, 35, num_samples),
                           np.where(categories == 1, np.random.uniform(40, 70, num_samples),
                                    np.random.uniform(75, 95, num_samples)))

    data['problem_solving'] = np.where(categories == 0, np.random.uniform(10, 35, num_samples),
                              np.where(categories == 1, np.random.uniform(40, 70, num_samples),
                                      np.random.uniform(75, 95, num_samples)))

    data['cultural_fit'] = np.where(categories == 0, np.random.uniform(10, 35, num_samples),
                           np.where(categories == 1, np.random.uniform(40, 70, num_samples),
                                   np.random.uniform(75, 95, num_samples)))

    data['overall'] = np.where(categories == 0, np.random.uniform(10, 35, num_samples),
                      np.where(categories == 1, np.random.uniform(40, 70, num_samples),
                              np.random.uniform(75, 95, num_samples)))

    data['classifier'] = categories

    print(f"Minimal emergency dataset generated with {len(data)} rows and {len(data.columns)} columns")
    print(f"Category distribution: Weak: {sum(categories == 0)}, Average: {sum(categories == 1)}, Strong: {sum(categories == 2)}")

    return data

def train_evaluation_models(data=None, retrain=False, verbose=True, use_gpu=True):
    """
    Train and save regression models for interview evaluation.

    Args:
        data: DataFrame with features and targets
        retrain: Whether to retrain models even if they exist
        verbose: Whether to print progress information
        use_gpu: Whether to use GPU acceleration if available

    Returns:
        Dictionary with trained models and scaler
    """
    if verbose:
        print("\n==== INTERVIEW EVALUATION MODEL TRAINING ====")

    # Check if models already exist and we're not forced to retrain
    if os.path.exists("interview_evaluation_models.pkl") and not retrain:
        if verbose:
            print("Interview evaluation models already exist.")
            print("Use retrain=True to force retraining.")
        try:
            model_package = joblib.load("interview_evaluation_models.pkl")
            return model_package
        except Exception as e:
            print(f"Error loading models: {e}")
            print("Will retrain models...")

    # Load data if not provided
    if data is None:
        if verbose:
            print("No data provided. Attempting to load interview data...")
        try:
            # Try to load existing processed data
            data = pd.read_csv("interview_data.csv")
            if verbose:
                print(f"Successfully loaded data with {len(data)} records")
        except Exception as e:
            if verbose:
                print(f"Error loading interview_data.csv: {e}")
                print("Generating sample data for demonstration...")

            try:
                # Import the necessary module for data generation
                import sys
                from pathlib import Path

                # Add parent directory to path if needed
                current_dir = Path(__file__).parent
                parent_dir = current_dir.parent
                if str(parent_dir) not in sys.path:
                    sys.path.append(str(parent_dir))

                # Import the data retrieval module
                try:
                    from section_5b_data_retrieval import get_interview_dataset

                    # Generate a moderate dataset for testing (5K samples)
                    num_samples = 50000  # Starting with 5K samples for initial testing
                    data = get_interview_dataset(num_samples=num_samples)

                    # Save the generated data for future use
                    data.to_csv(f"interview_data_sample_{num_samples}.csv", index=False)
                    if verbose:
                        print(f"Generated and saved sample data with {len(data)} records to interview_data_sample_{num_samples}.csv")
                except ImportError as e:
                    print(f"Error importing data retrieval module: {e}")
                    # Generate minimal sample data
                    data = generate_minimal_sample_data()
                    data.to_csv("interview_data_sample.csv", index=False)
                    if verbose:
                        print(f"Generated and saved minimal sample data with {len(data)} records to interview_data_sample.csv")
            except Exception as e:
                print(f"Error generating data: {e}")
                return None

    # Continue with the rest of the function for model training
    if verbose:
        print("Starting ML model training...")
        if use_gpu:
            print("Using GPU acceleration if available")

    # Check if models already exist and we don't want to retrain
    if not retrain and os.path.exists("interview_evaluation_models.pkl"):
        try:
            models = joblib.load("interview_evaluation_models.pkl")
            if verbose:
                print("Loaded existing models from interview_evaluation_models.pkl")
            return models
        except Exception as e:
            print(f"Error loading existing models: {e}")
            print("Will train new models instead")

    if data is None or len(data) == 0:
        print("No data available for training")
        return None

    if verbose:
        print(f"Training ML models on {len(data)} samples...")

    # Extract features and targets
    feature_columns = [col for col in data.columns if col.startswith('feature_')]
    target_columns = ['technical', 'communication', 'problem_solving', 'cultural_fit', 'overall']

    # Check which target columns are available
    available_targets = [col for col in target_columns if col in data.columns]

    if len(feature_columns) == 0:
        print("No feature columns found in data")
        return None

    if len(available_targets) == 0:
        print("No target columns found in data")
        return None

    if verbose:
        print(f"Using {len(feature_columns)} features for model training")
        print(f"Using {len(available_targets)} evaluation metrics as targets: {available_targets}")

    # Extract features
    X = data[feature_columns].copy()

    # Ensure all feature values are numeric
    for col in X.columns:
        if X[col].dtype == 'object':
            try:
                X[col] = pd.to_numeric(X[col], errors='coerce')
            except:
                print(f"Warning: Could not convert column {col} to numeric")
                # Drop the column if it can't be converted
                X = X.drop(columns=[col])

    # Fill NaN values with column means
    X = X.fillna(X.mean())

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

    # Dictionary to store trained models
    trained_models = {}

    # Train regression models for each target
    for target_name in available_targets:
        if verbose:
            print(f"\n=== Training models for {target_name} ===")

        # Get target values, ensuring they're numeric
        y = data[target_name].copy()
        if y.dtype == 'object':
            try:
                y = pd.to_numeric(y, errors='coerce')
            except:
                print(f"Warning: Could not convert target {target_name} to numeric")
                continue

        # Drop rows with missing target values
        valid_mask = ~y.isna()
        X_target = X_scaled[valid_mask].copy()
        y_target = y[valid_mask].copy()

        if len(y_target) == 0:
            print(f"No valid data for target {target_name}")
            continue

        # Split data for training and testing
        X_train, X_test, y_train, y_test = train_test_split(
            X_target, y_target, test_size=0.2, random_state=42
        )

        if verbose:
            print(f"Train set: {X_train.shape[0]} samples, Test set: {X_test.shape[0]} samples")

        # Train regression models
        best_model, _ = train_regression_models(
            X_train, X_test, y_train, y_test,
            target_name=target_name,
            use_gpu=use_gpu,
            verbose=verbose
        )

        # Store the best model
        trained_models[target_name] = best_model

    # Train classifier for Strong/Average/Weak if 'category' column exists
    if 'category' in data.columns:
        if verbose:
            print("\n=== Training classifier for Strong/Average/Weak categories ===")

        # Map categories to numerical values
        category_mapping = {'Strong': 2, 'Average': 1, 'Weak': 0}

        # Create a copy to avoid warnings
        data_for_class = data.copy()

        # Convert categories to numeric values
        if data_for_class['category'].dtype == 'object':
            # Map categorical values to numbers
            data_for_class['category_encoded'] = data_for_class['category'].map(category_mapping)

            # Check for invalid categories
            if data_for_class['category_encoded'].isna().any():
                invalid_cats = data_for_class.loc[data_for_class['category_encoded'].isna(), 'category'].unique()
                print(f"Warning: Found invalid categories: {invalid_cats}")
                print("These will be treated as missing values")
                # Fill NaN with a default value
                data_for_class = data_for_class.dropna(subset=['category_encoded'])

            # Ensure it's an integer
            data_for_class['category_encoded'] = data_for_class['category_encoded'].astype(int)

        # Use the same feature set as for regression
        X_class = X_scaled.loc[data_for_class.index].copy()
        y_class = data_for_class['category_encoded'].copy()

        # Train-test split
        X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
            X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
        )

        # Train the classifier
        classifier_model = train_classifier(
            X_train_class, X_test_class, y_train_class, y_test_class,
            use_gpu=use_gpu,
            verbose=verbose
        )

        # Store the classifier
        trained_models['classifier'] = classifier_model
        # Also store the classifier type (for prediction handling)
        trained_models['classifier_type'] = 'xgboost' if isinstance(classifier_model, xgb.Booster) else 'sklearn'

    # ==== 6.3: EVALUATING ML MODELS ====

    if verbose:
        print("\n=== Evaluating models on full dataset ===")

    # Try to run predictions on the full dataset
    try:
        # Convert the full dataset to DMatrix for XGBoost models
        X_dmatrix = xgb.DMatrix(X_scaled)

        # Predict using all models
        predictions = {}

        for target_name, model in trained_models.items():
            if target_name == 'classifier_type':
                continue  # Skip the type indicator

            if target_name == 'classifier':
                # Handle classifier predictions based on type
                if trained_models['classifier_type'] == 'xgboost':
                    # For XGBoost Booster, use DMatrix
                    class_preds_num = model.predict(X_dmatrix).astype(int)
                    # Map back to labels
                    class_mapping = {0: 'Weak', 1: 'Average', 2: 'Strong'}
                    class_preds = [class_mapping[pred] for pred in class_preds_num]
                else:
                    # For sklearn models
                    class_preds_num = model.predict(X_scaled)
                    # Map back to labels
                    class_mapping = {0: 'Weak', 1: 'Average', 2: 'Strong'}
                    class_preds = [class_mapping[pred] for pred in class_preds_num]

                predictions['category'] = class_preds

                if verbose:
                    # Show distribution of predictions
                    counts = {label: class_preds.count(label) for label in set(class_preds)}
                    print(f"    Classification results: {counts}")
            else:
                # Handle regression model predictions
                if isinstance(model, xgb.Booster):
                    # For XGBoost Booster, use DMatrix
                    target_preds = model.predict(X_dmatrix)
                else:
                    # For sklearn models
                    target_preds = model.predict(X_scaled)

                predictions[target_name] = target_preds

                if verbose:
                    # Show summary statistics of predictions
                    pred_mean = np.mean(target_preds)
                    pred_min = np.min(target_preds)
                    pred_max = np.max(target_preds)
                    print(f"    Score range: {pred_min:.2f} to {pred_max:.2f}, average: {pred_mean:.2f}")

    except Exception as e:
        print(f"Warning: Error during evaluation: {e}")
        print("This won't affect the saved models, but you may want to investigate")

    # ==== 6.4: SAVING ML MODELS ====

    # Package models with scaler for deployment
    model_package = {
        'models': trained_models,
        'scaler': scaler,
        'feature_columns': feature_columns,
        'target_columns': available_targets
    }

    try:
        # Save the model package
        joblib.dump(model_package, "interview_evaluation_models.pkl")
        if verbose:
            print("\nAll models saved successfully to interview_evaluation_models.pkl")
    except Exception as e:
        print(f"Error saving models: {e}")

    return model_package

# Function to make predictions with the trained models
def predict_with_models(data, model_package=None, verbose=True):
    """
    Make predictions using trained models

    Args:
        data: DataFrame with features
        model_package: Dictionary with trained models and scaler
        verbose: Whether to print progress information

    Returns:
        Dictionary of predictions
    """
    if verbose:
        print("Starting prediction process...")

    # Load models if not provided
    if model_package is None:
        if verbose:
            print("No model package provided, attempting to load from file...")
        try:
            model_package = joblib.load("interview_evaluation_models.pkl")
            if verbose:
                print("Successfully loaded models from interview_evaluation_models.pkl")
        except Exception as e:
            print(f"Error loading models: {e}")
            return None

    # Extract components from the package
    models = model_package['models']
    scaler = model_package['scaler']
    feature_columns = model_package['feature_columns']

    # Ensure all needed features are present
    for col in feature_columns:
        if col not in data.columns:
            data[col] = 0  # Default value for missing features

    # Extract required features in the correct order
    X = data[feature_columns].copy()

    # Ensure all feature values are numeric
    for col in X.columns:
        if not np.issubdtype(X[col].dtype, np.number):
            X[col] = X[col].astype(float)

    # Scale features
    X_scaled = scaler.transform(X)

    # Import XGBoost if needed for predictions
    try:
        import xgboost as xgb
        xgboost_available = True
    except ImportError:
        xgboost_available = False

    # Create DMatrix for XGBoost models if needed
    needs_dmatrix = False
    for model_name, model in models.items():
        if model_name != 'classifier_type' and isinstance(model, xgb.Booster):
            needs_dmatrix = True
            break

    if needs_dmatrix and xgboost_available:
        X_dmatrix = xgb.DMatrix(X_scaled)

    # Make predictions with all models
    predictions = {}
    for target_name, model in models.items():
        if target_name == 'classifier_type':
            continue  # Skip the type indicator

        if verbose:
            print(f"Predicting {target_name}...")

        if target_name == 'classifier':
            # Check classifier type
            if 'classifier_type' in models and models['classifier_type'] == 'xgboost':
                # For XGBoost booster
                class_preds_num = model.predict(X_dmatrix).astype(int)
                # Map back to labels
                class_mapping = {0: 'Weak', 1: 'Average', 2: 'Strong'}
                class_preds = [class_mapping[pred] for pred in class_preds_num]
                predictions['category'] = class_preds
            else:
                # For sklearn classifiers
                class_preds_num = model.predict(X_scaled)
                # Map back to labels
                class_mapping = {0: 'Weak', 1: 'Average', 2: 'Strong'}
                class_preds = [class_mapping[pred] for pred in class_preds_num]
                predictions['category'] = class_preds
        else:
            # Regression predictions
            if isinstance(model, xgb.Booster):
                # For XGBoost booster
                target_preds = model.predict(X_dmatrix)
            else:
                # For sklearn models
                target_preds = model.predict(X_scaled)

            predictions[target_name] = target_preds

    # Apply additional sentiment-based adjustment
    # Penalize inappropriate responses more heavily based on sentiment
    if 'feature_10' in data.columns:  # Check if sentiment feature exists
        sentiment_score = data['feature_10'].values[0]
        minimal_words = data['feature_0'].values[0] < 5  # Check if response is too short

        # Detect extremely negative or inappropriate responses
        # Lower sentiment with few words indicates potential inappropriate response
        if minimal_words and sentiment_score < 0.4:
            # Apply stronger penalty to all scores
            penalty_factor = 0.3
            for key in predictions:
                if key != 'category':  # Don't penalize the category
                    predictions[key] = np.array([max(20, score * penalty_factor) for score in predictions[key]])

    # Ensure all scores are within valid range (0-100)
    for key in predictions:
        if key != 'category':  # Don't adjust the category
            predictions[key] = np.array([max(0, min(100, score)) for score in predictions[key]])

    return predictions

# Example usage:
if __name__ == "__main__":
    print("ML Model Training Module")
    print("Starting direct execution of model training...")

    try:
        # Attempt to load data
        print("\nAttempting to load interview data...")
        try:
            data = pd.read_csv("interview_data.csv")
            print(f"Successfully loaded data with {len(data)} samples")
        except Exception as e:
            print(f"Error loading interview_data.csv: {e}")
            print("Generating sample data for demonstration...")

            # Create sample data if file doesn't exist
            np.random.seed(42)
            n_samples = 100000
            n_features = 10

            # Generate features
            features = np.random.randn(n_samples, n_features)
            feature_cols = [f'feature_{i}' for i in range(n_features)]

            # Generate target values
            technical = 0.7 * features[:, 0] + 0.3 * features[:, 1] + np.random.randn(n_samples) * 0.2
            communication = 0.5 * features[:, 2] + 0.5 * features[:, 3] + np.random.randn(n_samples) * 0.2
            problem_solving = 0.6 * features[:, 4] + 0.4 * features[:, 5] + np.random.randn(n_samples) * 0.2
            cultural_fit = 0.4 * features[:, 6] + 0.6 * features[:, 7] + np.random.randn(n_samples) * 0.2
            overall = 0.25 * technical + 0.25 * communication + 0.25 * problem_solving + 0.25 * cultural_fit

            # Scale to 0-100 range
            def scale_to_range(arr, min_val=0, max_val=100):
                return ((arr - arr.min()) / (arr.max() - arr.min())) * (max_val - min_val) + min_val

            technical = scale_to_range(technical)
            communication = scale_to_range(communication)
            problem_solving = scale_to_range(problem_solving)
            cultural_fit = scale_to_range(cultural_fit)
            overall = scale_to_range(overall)

            # Generate categories based on overall score
            categories = []
            for score in overall:
                if score >= 70:
                    categories.append('Strong')
                elif score >= 40:
                    categories.append('Average')
                else:
                    categories.append('Weak')

            # Create DataFrame
            data = pd.DataFrame(features, columns=feature_cols)
            data['technical'] = technical
            data['communication'] = communication
            data['problem_solving'] = problem_solving
            data['cultural_fit'] = cultural_fit
            data['overall'] = overall
            data['category'] = categories

            # Save sample data for future use
            try:
                data.to_csv("interview_data_sample.csv", index=False)
                print(f"Generated and saved sample data with {len(data)} records to interview_data_sample.csv")
            except:
                print("Note: Could not save sample data to file")

        # Train models
        print("\nTraining models...")
        # The global use_gpu variable was defined at the top of the file
        # It is automatically set based on GPU detection
        model_package = train_evaluation_models(
            data=data,
            retrain=True,
            verbose=True,
            use_gpu=use_gpu  # Use the global variable defined at the top of the file
        )

        if model_package:
            print("\nModel training completed successfully!")

    except Exception as e:
        print(f"\nAn error occurred during execution: {str(e)}")
        import traceback
        traceback.print_exc()

    print("\nExecution complete!")

TensorFlow version: 2.18.0
GPU devices available: 1
GPU details: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
PyTorch CUDA available: True
CUDA device count: 1
CUDA device name: Tesla T4
Optuna not available, will use RandomizedSearchCV instead
XGBoost available for advanced modeling
ML Model Training Module
Starting direct execution of model training...

Attempting to load interview data...
Error loading interview_data.csv: [Errno 2] No such file or directory: 'interview_data.csv'
Generating sample data for demonstration...
Generated and saved sample data with 100000 records to interview_data_sample.csv

Training models...

==== INTERVIEW EVALUATION MODEL TRAINING ====
Starting ML model training...
Using GPU acceleration if available
Training ML models on 100000 samples...
Using 10 features for model training
Using 5 evaluation metrics as targets: ['technical', 'communication', 'problem_solving', 'cultural_fit', 'overall']

=== Training models for technical ===
T

# Section 8: Interview Process

In [ ]:
"""# Section 8: Interview Process"""

#!/usr/bin/env python3
# Interview Process - Interactive interview evaluation tool

import os
import numpy as np
import pandas as pd
import joblib
import re
import time
from collections import Counter

# Try to import transformers for sentiment analysis
try:
    from transformers import pipeline
    TRANSFORMERS_AVAILABLE = True
    # Initialize sentiment analysis pipeline
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
except ImportError:
    TRANSFORMERS_AVAILABLE = False
    print("Transformers library not available. Using basic sentiment analysis.")

# Check if models exist, if not, try to train them
if not os.path.exists("interview_evaluation_models.pkl"):
    print("Interview evaluation models not found. Please run train_ml_models.py first.")
    print("Attempting to run training script...")
    try:
        import train_ml_models
        print("Training completed.")
    except Exception as e:
        print(f"Error training models: {e}")
        print("Please run train_ml_models.py manually before using this script.")
        exit(1)

# Load the trained models
try:
    print("Loading interview evaluation models...")
    model_package = joblib.load("interview_evaluation_models.pkl")
    print("Models loaded successfully!")
except Exception as e:
    print(f"Error loading models: {e}")
    exit(1)

def clear_screen():
    """Clear the terminal screen."""
    os.system('cls' if os.name == 'nt' else 'clear')

def typing_effect(text, speed=0.03):
    """Display text with a typing effect."""
    for char in text:
        print(char, end='', flush=True)
        time.sleep(speed)
    print()

def extract_features_from_text(text, language="english"):
    """
    Extract features from interview text response.

    Args:
        text: The user's response text
        language: The language of the response (english or arabic)

    Returns:
        Dictionary of features
    """
    # Ensure text is a string and not empty
    if text is None:
        text = ""
    text = str(text).strip()

    # Length-based features
    word_count = len(text.split())
    char_count = len(text)
    avg_word_length = char_count / max(word_count, 1)

    # Count sentence endings
    sentence_count = text.count('.') + text.count('!') + text.count('?')

    # Set default sentiment value
    sentiment_value = 0.5

    # Basic sentiment analysis using predefined word lists if transformers is not available
    positive_words = ["good", "great", "excellent", "amazing", "awesome", "fantastic", "positive",
                       "happy", "love", "enjoy", "thank", "appreciate", "enthusiastic", "success"]

    negative_words = ["bad", "poor", "terrible", "horrible", "awful", "negative", "sad", "hate",
                       "dislike", "angry", "upset", "fail", "disappointing", "never", "no", "not"]

    text_lower = text.lower()

    # Perform sentiment analysis
    if TRANSFORMERS_AVAILABLE and word_count > 3:
        try:
            # Use transformers pipeline for better sentiment analysis
            sentiment_result = sentiment_analyzer(text)[0]
            sentiment_score = sentiment_result['score']
            sentiment_label = sentiment_result['label']

            # Convert sentiment to numeric values (0-1 range)
            if sentiment_label == "POSITIVE":
                sentiment_value = sentiment_score
            else:
                sentiment_value = 1 - sentiment_score

        except Exception as e:
            print(f"Error in sentiment analysis: {e}")
            # Fallback to basic sentiment analysis
            positive_count = sum(1 for word in positive_words if word in text_lower)
            negative_count = sum(1 for word in negative_words if word in text_lower)
            sentiment_value = positive_count / (positive_count + negative_count + 1)
    else:
        # Basic sentiment analysis based on word matching
        positive_count = sum(1 for word in positive_words if word in text_lower)
        negative_count = sum(1 for word in negative_words if word in text_lower)
        sentiment_value = positive_count / (positive_count + negative_count + 1)

    # Technical keywords - basic example (would be more extensive in real app)
    technical_keywords = [
        "algorithm", "data structure", "code", "programming", "software",
        "development", "database", "api", "function", "class", "object",
        "method", "variable", "framework", "library", "interface", "architecture",
        "test", "debug", "deploy", "cloud", "server", "client", "network"
    ]

    # Communication keywords
    communication_keywords = [
        "communicate", "team", "collaborate", "explain", "present",
        "discuss", "share", "meetings", "documentation", "report",
        "clarity", "articulate", "express", "listen", "feedback"
    ]

    # Problem-solving keywords
    problem_solving_keywords = [
        "solve", "solution", "analyze", "optimize", "improve", "debug",
        "troubleshoot", "fix", "enhance", "approach", "method", "strategy",
        "plan", "design", "implement", "test"
    ]

    # Cultural fit keywords
    cultural_fit_keywords = [
        "team", "culture", "values", "mission", "collaborate", "work ethic",
        "adaptable", "flexible", "learn", "growth", "positive", "attitude",
        "initiative", "proactive", "responsible", "accountable"
    ]

    # Negative response indicators
    negative_phrases = [
        "i don't know", "no idea", "not sure", "never heard", "no experience",
        "i haven't", "can't answer", "don't understand", "no clue", "not familiar"
    ]

    # Check for minimal or negative responses
    text_lower = text.lower()

    # Detect if response is minimal or negative
    is_minimal_response = word_count <= 5
    has_negative_phrase = any(phrase in text_lower for phrase in negative_phrases)

    # Apply severe scoring penalty for minimal or negative responses
    minimal_response_penalty = 0.1 if is_minimal_response or has_negative_phrase else 1.0

    # Count occurrences of each keyword type
    tech_count = sum(1 for word in technical_keywords if word in text_lower)
    comm_count = sum(1 for word in communication_keywords if word in text_lower)
    prob_count = sum(1 for word in problem_solving_keywords if word in text_lower)
    cult_count = sum(1 for word in cultural_fit_keywords if word in text_lower)

    # Ratios
    tech_ratio = tech_count / max(word_count, 1)
    comm_ratio = comm_count / max(word_count, 1)
    prob_ratio = prob_count / max(word_count, 1)
    cult_ratio = cult_count / max(word_count, 1)

    # Simple confidence score based on word count and keyword usage
    base_confidence = min(100, (word_count / 50) * 100 + (tech_count + comm_count) * 5)
    confidence_score = base_confidence * minimal_response_penalty

    # Calculate quality score based on response length and complexity
    quality_score = min(100, word_count * 2) * minimal_response_penalty

    # Adjust scores for minimal or negative responses
    if is_minimal_response or has_negative_phrase:
        tech_count *= 0.1
        comm_count *= 0.1
        prob_count *= 0.1
        cult_count *= 0.1
        tech_ratio *= 0.1
        comm_ratio *= 0.1
        prob_ratio *= 0.1
        cult_ratio *= 0.1

    # Create feature dictionary (matching expected model feature names)
    features = {
        'feature_0': float(word_count * minimal_response_penalty),         # Word count
        'feature_1': float(char_count * minimal_response_penalty),         # Character count
        'feature_2': float(avg_word_length),                              # Average word length
        'feature_3': float(sentence_count * minimal_response_penalty),     # Number of sentences
        'feature_4': float(tech_count),                                   # Technical keyword count
        'feature_5': float(comm_count),                                   # Communication keyword count
        'feature_6': float(prob_count),                                   # Problem-solving keyword count
        'feature_7': float(cult_count),                                   # Cultural fit keyword count
        'feature_8': float(confidence_score),                             # Confidence score
        'feature_9': float((tech_ratio + comm_ratio + prob_ratio + cult_ratio) * 25 * minimal_response_penalty),  # Overall keyword ratio
        'feature_10': float(sentiment_value),                                # Sentiment score
    }

    return features

def predict_response_scores(features):
    """
    Predict scores for an interview response using loaded models.

    Args:
        features: Dictionary of features extracted from response

    Returns:
        Dictionary of predicted scores and category
    """
    try:
        # Extract word count from features (stored as feature_0)
        word_count = features.get('feature_0', 0)

        # Convert features to DataFrame
        features_df = pd.DataFrame([features])

        # Ensure all required features (feature_0 through feature_10) are present
        for i in range(11):
            feature_name = f'feature_{i}'
            if feature_name not in features_df.columns:
                print(f"Warning: {feature_name} not found in extracted features. Using default value 0.")
                features_df[feature_name] = 0

        # Ensure all feature values are numeric
        for col in features_df.columns:
            features_df[col] = pd.to_numeric(features_df[col], errors='coerce').fillna(0)

        # Make predictions
        predictions = predict_with_models(features_df, model_package, verbose=False)

        # If predictions are None, return a default response
        if predictions is None:
            print("Warning: Model prediction failed. Returning default scores.")
            # For a minimal response like "idk", use appropriate low scores and Weak category
            if word_count < 5:
                return {
                    'communication': np.array([20.0]),
                    'technical': np.array([20.0]),
                    'problem_solving': np.array([20.0]),
                    'cultural_fit': np.array([20.0]),
                    'overall': np.array([20.0]),
                    'category': np.array(['Weak'])
                }
            else:
                return {
                    'communication': np.array([50.0]),
                    'technical': np.array([50.0]),
                    'problem_solving': np.array([50.0]),
                    'cultural_fit': np.array([50.0]),
                    'overall': np.array([50.0]),
                    'category': np.array(['Average'])
                }

        # Ensure all predictions are in array format
        for key in predictions:
            if key != 'category' and not isinstance(predictions[key], (list, np.ndarray)):
                predictions[key] = np.array([predictions[key]])
            elif key == 'category' and not isinstance(predictions[key], (list, np.ndarray)):
                predictions[key] = np.array([predictions[key]])

        return predictions

    except Exception as e:
        print(f"Error in predict_response_scores: {e}")
        # Return a default set of scores based on word count
        if word_count < 5:
            return {
                'communication': np.array([20.0]),
                'technical': np.array([20.0]),
                'problem_solving': np.array([20.0]),
                'cultural_fit': np.array([20.0]),
                'overall': np.array([20.0]),
                'category': np.array(['Weak'])
            }
        else:
            return {
                'communication': np.array([50.0]),
                'technical': np.array([50.0]),
                'problem_solving': np.array([50.0]),
                'cultural_fit': np.array([50.0]),
                'overall': np.array([50.0]),
                'category': np.array(['Average'])
            }

def predict_with_models(data, model_package, verbose=False):
    """Simplified version of the predict_with_models function from train_ml_models.py"""
    try:
        # Extract components from the package
        models = model_package['models']
        scaler = model_package['scaler']
        feature_columns = model_package['feature_columns']

        # Ensure all needed features are present
        for col in feature_columns:
            if col not in data.columns:
                if verbose:
                    print(f"Adding missing feature column: {col}")
                data[col] = 0  # Default value for missing features

        # Extract required features in the correct order
        X = data[feature_columns].copy()

        # Ensure all feature values are numeric
        for col in X.columns:
            if not np.issubdtype(X[col].dtype, np.number):
                X[col] = X[col].astype(float)

        # Scale features
        X_scaled = scaler.transform(X)

        # Import XGBoost if needed for predictions
        try:
            import xgboost as xgb
            xgboost_available = True
        except ImportError:
            xgboost_available = False

        # Create DMatrix for XGBoost models if needed
        needs_dmatrix = False
        X_dmatrix = None

        for model_name, model in models.items():
            if model_name != 'classifier_type' and isinstance(model, xgb.Booster):
                needs_dmatrix = True
                break

        if needs_dmatrix and xgboost_available:
            try:
                X_dmatrix = xgb.DMatrix(X_scaled)
            except Exception as e:
                print(f"Error creating DMatrix: {e}")
                return None

        # Make predictions with all models
        predictions = {}
        for target_name, model in models.items():
            if target_name == 'classifier_type':
                continue  # Skip the type indicator

            try:
                if target_name == 'classifier':
                    # Check classifier type
                    if 'classifier_type' in models and models['classifier_type'] == 'xgboost':
                        if X_dmatrix is None:
                            predictions['category'] = 'Average'  # Default if DMatrix failed
                            continue
                        # For XGBoost booster
                        class_preds_num = model.predict(X_dmatrix)[0]
                        # Map back to labels
                        class_mapping = {0: 'Weak', 1: 'Average', 2: 'Strong'}
                        predictions['category'] = class_mapping[int(class_preds_num)]
                    else:
                        # For sklearn classifiers
                        class_pred_num = model.predict(X_scaled)[0]
                        # Map back to labels
                        class_mapping = {0: 'Weak', 1: 'Average', 2: 'Strong'}
                        predictions['category'] = class_mapping[int(class_pred_num)]
                else:
                    # Regression predictions
                    if isinstance(model, xgb.Booster):
                        if X_dmatrix is None:
                            predictions[target_name] = 50.0  # Default score if DMatrix failed
                            continue
                        # For XGBoost booster
                        predictions[target_name] = float(model.predict(X_dmatrix)[0])
                    else:
                        # For sklearn models
                        predictions[target_name] = float(model.predict(X_scaled)[0])
            except Exception as e:
                print(f"Error predicting {target_name}: {e}")
                # Set a default prediction value on error
                if target_name == 'classifier':
                    predictions['category'] = 'Average'
                else:
                    predictions[target_name] = 50.0

        # Apply additional sentiment-based adjustment
        # Penalize inappropriate responses more heavily based on sentiment
        if 'feature_10' in data.columns:  # Check if sentiment feature exists
            sentiment_score = data['feature_10'].values[0]
            minimal_words = data['feature_0'].values[0] < 5  # Check if response is too short

            # Detect extremely negative or inappropriate responses
            # Lower sentiment with few words indicates potential inappropriate response
            if minimal_words and sentiment_score < 0.4:
                # Apply stronger penalty to all scores
                penalty_factor = 0.3
                for key in predictions:
                    if key != 'category':  # Don't penalize the category
                        predictions[key] = max(20, predictions[key] * penalty_factor)

        # Ensure all scores are within valid range (0-100)
        for key in predictions:
            if key != 'category':  # Don't adjust the category
                predictions[key] = max(0, min(100, predictions[key]))

        # Determine category based on overall score if not already set or if predictions are very low
        overall_score = predictions.get('overall', 0)
        minimal_response = data['feature_0'].values[0] < 5  # Check if response is too short

        # Override category based on overall score for very low scores or minimal responses
        if minimal_response or overall_score < 30:
            predictions['category'] = 'Weak'
        elif 'category' not in predictions:
            # Set category based on overall score
            if overall_score >= 75:
                predictions['category'] = 'Strong'
            elif overall_score >= 40:
                predictions['category'] = 'Average'
            else:
                predictions['category'] = 'Weak'

        return predictions

    except Exception as e:
        print(f"Error in predict_with_models: {e}")
        return None

def format_score(score):
    """Format a numerical score for display."""
    return f"{score:.1f}/100"

def get_score_emoji(score):
    """Get an appropriate emoji for a score."""
    if score >= 80:
        return "🔥"
    elif score >= 60:
        return "👍"
    elif score >= 40:
        return "👌"
    else:
        return "🤔"

def get_category_description(category):
    """Get a description for a category."""
    if category == 'Strong':
        return "Excellent performance! You demonstrate strong skills and understanding."
    elif category == 'Average':
        return "Solid performance. You show competence but could improve in some areas."
    else:  # Weak
        return "You need to develop your skills further in this area."

def display_results(predictions, question):
    """Display the evaluation results to the user."""
    clear_screen()

    # Safely extract prediction values, handle both arrays and single values
    def safe_extract(pred_dict, key):
        if key not in pred_dict:
            return 50.0 if key != 'category' else 'Average'

        value = pred_dict[key]
        if isinstance(value, (list, np.ndarray)) and len(value) > 0:
            return value[0]
        return value

    # Get prediction values
    technical = safe_extract(predictions, 'technical')
    communication = safe_extract(predictions, 'communication')
    problem_solving = safe_extract(predictions, 'problem_solving')
    cultural_fit = safe_extract(predictions, 'cultural_fit')
    overall = safe_extract(predictions, 'overall')
    category = safe_extract(predictions, 'category')

    # Make sure numerical values are floats
    technical = float(technical) if isinstance(technical, (int, float, np.number)) else 50.0
    communication = float(communication) if isinstance(communication, (int, float, np.number)) else 50.0
    problem_solving = float(problem_solving) if isinstance(problem_solving, (int, float, np.number)) else 50.0
    cultural_fit = float(cultural_fit) if isinstance(cultural_fit, (int, float, np.number)) else 50.0
    overall = float(overall) if isinstance(overall, (int, float, np.number)) else 50.0

    # Ensure category is a string
    if isinstance(category, (list, np.ndarray)) and len(category) > 0:
        category = category[0]
    category = str(category) if category is not None else "Average"

    typing_effect("\n📊 INTERVIEW RESPONSE EVALUATION 📊\n", speed=0.01)
    time.sleep(0.5)

    print(f"Question: {question}\n")

    typing_effect("Your performance metrics:", speed=0.02)
    time.sleep(0.3)

    # Display individual scores with emojis
    print(f"  Technical Proficiency:  {format_score(technical)} {get_score_emoji(technical)}")
    time.sleep(0.2)
    print(f"  Communication Skills:   {format_score(communication)} {get_score_emoji(communication)}")
    time.sleep(0.2)
    print(f"  Problem-Solving:        {format_score(problem_solving)} {get_score_emoji(problem_solving)}")
    time.sleep(0.2)
    print(f"  Cultural Fit:           {format_score(cultural_fit)} {get_score_emoji(cultural_fit)}")
    time.sleep(0.2)
    print(f"  Overall Score:          {format_score(overall)} {get_score_emoji(overall)}")
    time.sleep(0.2)

    # Display category with description
    print("\nOverall Classification:")
    if category == "Strong":
        typing_effect(f"  ⭐ {category} ⭐", speed=0.02)
    elif category == "Average":
        typing_effect(f"  ✓ {category} ✓", speed=0.02)
    else:
        typing_effect(f"  △ {category} △", speed=0.02)

    print(f"  {get_category_description(category)}")

    print("\nAreas for improvement:")
    scores = {
        "Technical knowledge": technical,
        "Communication clarity": communication,
        "Problem-solving approach": problem_solving,
        "Team and culture fit": cultural_fit
    }

    # Sort scores to find weakest areas
    sorted_scores = sorted(scores.items(), key=lambda x: x[1])

    # Show suggestions for the two weakest areas
    for area, score in sorted_scores[:2]:
        print(f"  • {area} ({format_score(score)})")
        if area == "Technical knowledge":
            print("    Consider reviewing key concepts and practicing coding problems.")
        elif area == "Communication clarity":
            print("    Work on articulating your thoughts more clearly and concisely.")
        elif area == "Problem-solving approach":
            print("    Practice breaking down complex problems into smaller steps.")
        else:  # Cultural fit
            print("    Focus on demonstrating teamwork and alignment with company values.")

def run_interview(max_questions=3, all_results=None, question_num=1):
    """Run the interactive interview process.

    Args:
        max_questions: Maximum number of questions to ask
        all_results: List to store all predictions
        question_num: Current question number
    """
    # Initialize all_results if this is the first question
    if all_results is None:
        all_results = []

    clear_screen()

    # Only show the welcome message on the first question
    if question_num == 1:
        typing_effect("🤖 Welcome to the AI Interview Evaluation System! 🤖", speed=0.02)
        typing_effect("I'll ask you a few interview questions, analyze your responses,", speed=0.02)
        typing_effect("and provide feedback on your performance.", speed=0.02)
        typing_effect(f"You will answer {max_questions} questions in total.", speed=0.02)
        print()

    # List of interview questions
    questions = [
        "Tell me about yourself and your experience in software engineering.",
        "Describe a challenging project you worked on and how you overcame obstacles.",
        "How do you approach debugging a complex issue in your code?",
        "Explain how you would design a scalable web application architecture.",
        "How do you stay updated with the latest technologies in your field?",
        "Describe a situation where you had to work with a difficult team member.",
        "What's your experience with agile development methodologies?",
        "How would you optimize a slow-performing database query?",
        "Tell me about a time when you had to learn a new technology quickly."
    ]

    # Randomly select a question
    import random
    question = random.choice(questions)

    typing_effect(f"\nQuestion {question_num}/{max_questions}: {question}", speed=0.03)
    print("\nYour response (type your answer and press Enter when done):")

    # Get user's response
    response = input("> ")

    if not response.strip():
        typing_effect("I notice your response is empty. Let's try again with a more detailed answer.", speed=0.03)
        response = input("> ")

    print("\nAnalyzing your response...")
    time.sleep(1)

    # Process the response
    features = extract_features_from_text(response)

    # Redirect stderr to hide error messages from the user
    import sys
    import io
    original_stderr = sys.stderr
    sys.stderr = io.StringIO()  # Redirect stderr to a string buffer

    try:
        # Make predictions
        predictions = predict_response_scores(features)

        # Store question and predictions for summary
        result = {
            'question': question,
            'response': response,
            'predictions': predictions
        }
        all_results.append(result)

        # Display results
        display_results(predictions, question)
    finally:
        # Restore stderr
        sys.stderr = original_stderr

    # Check if we've reached the maximum number of questions
    if question_num < max_questions:
        print(f"\nMoving on to question {question_num + 1} of {max_questions}...")
        print("Press Enter to continue.")
        input()
        # Continue to next question
        run_interview(max_questions, all_results, question_num + 1)
    else:
        # Show final summary after all questions
        display_summary(all_results)
        typing_effect("\nThank you for completing the AI Interview Evaluation!", speed=0.02)
        typing_effect("Good luck with your interviews! 👋", speed=0.02)

def display_summary(all_results):
    """Display a summary of all interview responses.

    Args:
        all_results: List of dictionaries containing questions, responses, and predictions
    """
    clear_screen()
    typing_effect("\n📊 INTERVIEW EVALUATION SUMMARY 📊\n", speed=0.01)
    time.sleep(0.5)

    typing_effect(f"You answered {len(all_results)} questions in total.\n", speed=0.02)
    time.sleep(0.3)

    # Calculate average scores
    avg_technical = 0
    avg_communication = 0
    avg_problem_solving = 0
    avg_cultural_fit = 0
    avg_overall = 0
    categories = []

    for i, result in enumerate(all_results):
        predictions = result['predictions']

        # Safely extract values
        def safe_extract(pred_dict, key):
            if key not in pred_dict:
                return 50.0 if key != 'category' else 'Average'

            value = pred_dict[key]
            if isinstance(value, (list, np.ndarray)) and len(value) > 0:
                return value[0]
            return value

        # Extract scores
        technical = float(safe_extract(predictions, 'technical'))
        communication = float(safe_extract(predictions, 'communication'))
        problem_solving = float(safe_extract(predictions, 'problem_solving'))
        cultural_fit = float(safe_extract(predictions, 'cultural_fit'))
        overall = float(safe_extract(predictions, 'overall'))
        category = str(safe_extract(predictions, 'category'))

        # Add to averages
        avg_technical += technical
        avg_communication += communication
        avg_problem_solving += problem_solving
        avg_cultural_fit += cultural_fit
        avg_overall += overall
        categories.append(category)

        # Display brief summary of each question
        print(f"Question {i+1}: {result['question'][:60]}...")
        print(f"  Overall Score: {format_score(overall)} {get_score_emoji(overall)}")
        print(f"  Classification: {category}\n")

    # Calculate averages
    num_results = len(all_results)
    if num_results > 0:
        avg_technical /= num_results
        avg_communication /= num_results
        avg_problem_solving /= num_results
        avg_cultural_fit /= num_results
        avg_overall /= num_results

    # Determine overall category
    category_values = {'Strong': 3, 'Average': 2, 'Weak': 1}
    category_counts = {'Strong': 0, 'Average': 0, 'Weak': 0}

    for cat in categories:
        if cat in category_counts:
            category_counts[cat] += 1

    # Determine most frequent category
    final_category = max(category_counts.items(), key=lambda x: x[1])[0]

    # Display overall averages
    typing_effect("\n📈 OVERALL PERFORMANCE 📈\n", speed=0.01)
    time.sleep(0.5)

    print(f"Technical Proficiency:  {format_score(avg_technical)} {get_score_emoji(avg_technical)}")
    print(f"Communication Skills:   {format_score(avg_communication)} {get_score_emoji(avg_communication)}")
    print(f"Problem-Solving:        {format_score(avg_problem_solving)} {get_score_emoji(avg_problem_solving)}")
    print(f"Cultural Fit:           {format_score(avg_cultural_fit)} {get_score_emoji(avg_cultural_fit)}")
    print(f"Overall Interview Score: {format_score(avg_overall)} {get_score_emoji(avg_overall)}")

    print("\nFinal Classification:")
    if final_category == "Strong":
        typing_effect(f"  ⭐ {final_category} ⭐", speed=0.02)
    elif final_category == "Average":
        typing_effect(f"  ✓ {final_category} ✓", speed=0.02)
    else:
        typing_effect(f"  △ {final_category} △", speed=0.02)

    print(f"  {get_category_description(final_category)}")

    # Career advice based on overall performance
    print("\nKey takeaways and advice:")

    # Find weakest and strongest areas
    areas = {
        "Technical knowledge": avg_technical,
        "Communication clarity": avg_communication,
        "Problem-solving approach": avg_problem_solving,
        "Team and culture fit": avg_cultural_fit
    }

    sorted_areas = sorted(areas.items(), key=lambda x: x[1])
    weakest_area = sorted_areas[0]
    strongest_area = sorted_areas[-1]

    print(f"  • Your strongest area is {strongest_area[0]} ({format_score(strongest_area[1])}).")
    print(f"  • Focus on improving {weakest_area[0]} ({format_score(weakest_area[1])}).")

    if avg_overall >= 80:
        print("  • You show strong interview skills. Consider applying for more challenging positions.")
    elif avg_overall >= 60:
        print("  • You have a good foundation. Practice more mock interviews to boost confidence.")
    else:
        print("  • Continue practicing interview scenarios and focus on your weaker areas.")

    print("\nRecommended next steps:")
    print("  1. Review and practice questions in your weaker areas")
    print("  2. Record yourself answering questions to analyze your communication")
    print("  3. Seek feedback from industry professionals if possible")
    print("  4. Consider another practice session focusing on your weakest topics")

if __name__ == "__main__":
    run_interview()

Device set to use cpu


Loading interview evaluation models...
Models loaded successfully!
🤖 Welcom

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:41:18] WARNING: /workspace/src/gbm/gbtree.cc:363: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:41:18] WARNING: /workspace/src/gbm/gbtree.cc:388: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:41:18] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserW

e to the AI Interview Evaluation System! 🤖
I'll ask you a few interview questions, analyze your responses,
and provide feedback on your performance.
You will answer 3 questions in total.


Question 1/3: Describe a situation where you had to work with a difficult team member.

Your response (type your answer and press Enter when done):
> i slapped them on the face

Analyzing your response...
Error predicting overall: training data did not have the following fields: feature_0, feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7, feature_8, feature_9
Error predicting classifier: training data did not have the following fields: feature_0, feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7, feature_8, feature_9

📊 INTERVIEW RESPONSE EVALUATION 📊

Question: Describe a situation where you had to work with a difficult team member.

Your performance metrics:
  Technical Proficiency:  100.0/100 🔥
  Communication Skills:   80.0/100 🔥
  Problem-Solv

# Section 9: Colab Utilities

In [ ]:
# ============== SECTION 9: COLAB UTILITIES ==============
# These utilities are helpful when running in Google Colab

def colab_utils_setup():
    """Set up utilities for Google Colab environment"""
    if not USING_COLAB:
        print("Not running in Google Colab, skipping utilities setup.")
        return False

    try:
        from google.colab import files
        import matplotlib.pyplot as plt
        from IPython.display import display, HTML

        # Define utility functions for Colab
        def download_file(filename):
            """Trigger file download in Colab"""
            try:
                files.download(filename)
                print(f"Download initiated for {filename}")
                return True
            except Exception as e:
                print(f"Error downloading file: {e}")
                return False

        def upload_file():
            """Handle file upload in Colab"""
            try:
                uploaded = files.upload()
                print(f"Uploaded {len(uploaded)} files")
                return list(uploaded.keys())
            except Exception as e:
                print(f"Error uploading files: {e}")
                return []

        def display_html(html_content):
            """Display rich HTML in Colab"""
            display(HTML(html_content))

        def display_evaluation_chart(results):
            """Display a radar chart for interview evaluation"""
            categories = ['Technical', 'Communication', 'Problem Solving', 'Cultural Fit']
            values = [
                results.get('technical_score', 0)/10,
                results.get('communication_score', 0)/10,
                results.get('problem_solving', 0)/10,
                results.get('cultural_fit', 0)/10
            ]

            # Create radar chart
            fig = plt.figure(figsize=(10, 6))
            ax = fig.add_subplot(111, polar=True)

            # Set the angles for each category
            angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
            # Make the plot circular
            values = values + [values[0]]
            angles = angles + [angles[0]]
            categories = categories + [categories[0]]

            # Plot and fill
            ax.plot(angles, values, 'o-', linewidth=2)
            ax.fill(angles, values, alpha=0.25)

            # Set category labels
            ax.set_thetagrids(np.degrees(angles[:-1]), categories[:-1])

            # Set radial ticks and labels
            ax.set_rlabel_position(0)
            ax.set_rticks([0, 2.5, 5, 7.5, 10])
            ax.set_rlim(0, 10)

            # Add title
            recommendation = results.get('recommendation', 'No recommendation')
            plt.title(f"Interview Evaluation\nOverall Score: {results.get('overall_score', 0)/10:.1f}/10 - {recommendation}",
                    size=15, color='blue', y=1.1)

            plt.tight_layout()
            plt.show()

        # Add these functions to globals
        globals()['download_file'] = download_file
        globals()['upload_file'] = upload_file
        globals()['display_html'] = display_html
        globals()['display_evaluation_chart'] = display_evaluation_chart

        print("✅ Colab utilities successfully configured")
        return True

    except Exception as e:
        print(f"Error setting up Colab utilities: {e}")
        return False

# Set up Colab utilities if we're in Colab
if USING_COLAB:
    colab_utils_setup()

    # Modify the run_interview function to use Colab utilities when available
    original_run_interview = run_interview

    def enhanced_colab_run_interview():
        """Enhanced version of run_interview with Colab-specific features"""
        results = original_run_interview()

        # If we have evaluation results and the display function exists
        if results and 'display_evaluation_chart' in globals():
            print("\nGenerating visual evaluation chart...")
            display_evaluation_chart(results)

        return results

    # Replace the original function with our enhanced version
    run_interview = enhanced_colab_run_interview

# Function to analyze and visualize the datasets in Colab
def explore_datasets():
    """Analyze and visualize interview questions and evaluations datasets in Colab"""
    if not USING_COLAB:
        print("This function is designed for Google Colab environment")
        return

    try:
        # Import visualization libraries
        import matplotlib.pyplot as plt
        import seaborn as sns
        sns.set_style("whitegrid")

        # Load datasets
        print("Loading datasets...")
        questions_df = pd.read_csv("interview_questions.csv")
        evaluations_df = pd.read_csv("evaluation_scores.csv")

        print(f"Interview Questions: {len(questions_df)} records")
        print(f"Evaluation Scores: {len(evaluations_df)} records")

        # Display dataset heads
        print("\n--- Interview Questions Sample ---")
        display(questions_df.head(3))

        print("\n--- Evaluation Scores Sample ---")
        display(evaluations_df.head(3))

        # Visualize question distributions
        plt.figure(figsize=(16, 10))

        # Plot profession distribution
        plt.subplot(2, 2, 1)
        profession_counts = questions_df['profession'].value_counts()
        sns.barplot(x=profession_counts.index, y=profession_counts.values)
        plt.title('Distribution of Professions')
        plt.xticks(rotation=90)
        plt.ylabel('Count')

        # Plot question type distribution
        plt.subplot(2, 2, 2)
        q_type_counts = questions_df['question_type'].value_counts()
        sns.barplot(x=q_type_counts.index, y=q_type_counts.values)
        plt.title('Distribution of Question Types')
        plt.xticks(rotation=45)
        plt.ylabel('Count')

        # Plot difficulty level distribution
        plt.subplot(2, 2, 3)
        difficulty_counts = questions_df['difficulty_level'].value_counts()
        sns.barplot(x=difficulty_counts.index, y=difficulty_counts.values)
        plt.title('Distribution of Difficulty Levels')
        plt.ylabel('Count')

        # Histogram of evaluation scores
        plt.subplot(2, 2, 4)
        sns.histplot(evaluations_df['score'], bins=20, kde=True)
        plt.title('Distribution of Evaluation Scores')
        plt.xlabel('Score')
        plt.ylabel('Frequency')

        plt.tight_layout()
        plt.show()

        # Show correlation heatmap for merged data
        print("\nMerging datasets for correlation analysis...")
        merged_df = pd.merge(evaluations_df, questions_df, on='question_id')

        # Convert categorical variables to numeric for correlation
        merged_df = pd.get_dummies(merged_df, columns=['profession', 'question_type', 'difficulty_level'])
        merged_df['evaluation_date'] = pd.to_datetime(merged_df['evaluation_date'])
        merged_df['month'] = merged_df['evaluation_date'].dt.month
        merged_df['year'] = merged_df['evaluation_date'].dt.year

        # Select numeric columns for correlation
        numeric_cols = merged_df.select_dtypes(include=['number']).columns

        # Plot correlation heatmap
        plt.figure(figsize=(12, 10))
        corr_matrix = merged_df[numeric_cols].corr()
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        sns.heatmap(corr_matrix, mask=mask, annot=False, cmap='coolwarm', center=0)
        plt.title('Correlation Matrix of Dataset Features')
        plt.tight_layout()
        plt.show()

        # Interactive plot for score distribution by profession
        plt.figure(figsize=(14, 6))
        sns.boxplot(x='profession', y='score', data=merged_df)
        plt.title('Score Distribution by Profession')
        plt.xticks(rotation=90)
        plt.show()

        # Interactive plot for score distribution by question type
        plt.figure(figsize=(14, 6))
        sns.boxplot(x='question_type', y='score', data=merged_df)
        plt.title('Score Distribution by Question Type')
        plt.show()

        # Interactive plot for score distribution by difficulty level
        plt.figure(figsize=(14, 6))
        sns.boxplot(x='difficulty_level', y='score', data=merged_df)
        plt.title('Score Distribution by Difficulty Level')
        plt.show()

        print("Datasets exploration complete!")

    except FileNotFoundError:
        print("Error: Dataset files not found. Make sure you've generated the datasets first.")
    except Exception as e:
        print(f"Error exploring datasets: {e}")

# Add this function to Colab utilities if we're in Colab
if USING_COLAB and 'colab_utils_setup' in globals():
    globals()['explore_datasets'] = explore_datasets
    print("✅ Dataset exploration utility added. Call explore_datasets() to analyze your datasets.")

# Section 10: Extra UI

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, clear_output
import random
import sys
import os
from sklearn.metrics import confusion_matrix

# Check if running in Google Colab
USING_COLAB = 'google.colab' in sys.modules

# No need to import functions from section_8 as they should be available
# in the global namespace when running as cells in Colab

# Define fallback functions only if they don't exist in the global namespace
if 'format_score' not in globals():
    def format_score(score):
        return f"{score:.1f}/10"

if 'get_score_emoji' not in globals():
    def get_score_emoji(score):
        if score >= 80: return "🌟"
        elif score >= 60: return "✅"
        elif score >= 40: return "⚠️"
        else: return "❌"

if 'get_category_description' not in globals():
    def get_category_description(category):
        descriptions = {
            "Expert": "You demonstrate exceptional knowledge and communication skills.",
            "Proficient": "You show strong understanding and good communication.",
            "Competent": "You have a solid foundation but could improve in some areas.",
            "Developing": "You show some understanding but need significant improvement.",
            "Novice": "You need to develop fundamental knowledge and skills in this area.",
            "Average": "Your response shows average competency across all dimensions."
        }
        return descriptions.get(category, "Your response received a standard evaluation.")

if 'clear_screen' not in globals():
    def clear_screen():
        if not USING_COLAB:
            os.system('cls' if os.name == 'nt' else 'clear')

# ============================
# COLAB UTILITIES
# ============================

def setup_colab_environment():
    """Set up the Google Colab environment with necessary libraries and styling."""
    if not USING_COLAB:
        print("Not running in Google Colab. Skipping environment setup.")
        return False

    try:
        from google.colab import files

        # Set custom styling for better UI
        custom_css = """
        <style>
            .interview-header {
                background-color: #4285F4;
                color: white;
                padding: 15px;
                border-radius: 5px;
                margin-bottom: 20px;
                text-align: center;
                font-size: 24px;
                font-weight: bold;
            }

            .question-box {
                background-color: #f8f9fa;
                border-left: 5px solid #4285F4;
                padding: 10px 15px;
                margin: 20px 0;
                border-radius: 0 5px 5px 0;
                font-size: 18px;
            }

            .response-box {
                background-color: #e8f0fe;
                border: 1px solid #4285F4;
                padding: 15px;
                margin: 15px 0;
                border-radius: 5px;
                min-height: 100px;
            }

            .result-card {
                background-color: white;
                border: 1px solid #dadce0;
                border-radius: 8px;
                padding: 20px;
                margin: 20px 0;
                box-shadow: 0 1px 2px rgba(60,64,67,0.3);
            }

            .metric-row {
                display: flex;
                justify-content: space-between;
                margin: 10px 0;
                padding: 5px 0;
                border-bottom: 1px solid #f1f3f4;
            }

            .metric-label {
                font-weight: bold;
                color: #4285F4;
            }

            .progress-container {
                width: 100%;
                background-color: #e0e0e0;
                border-radius: 5px;
                margin: 5px 0;
            }

            .progress-bar {
                height: 20px;
                border-radius: 5px;
                text-align: center;
                color: white;
                font-weight: bold;
            }

            .feedback-box {
                background-color: #f1f3f4;
                border-left: 5px solid #fbbc04;
                padding: 15px;
                margin-top: 20px;
                border-radius: 0 5px 5px 0;
            }

            .summary-title {
                text-align: center;
                font-size: 22px;
                font-weight: bold;
                margin: 20px 0;
                color: #4285F4;
                border-bottom: 2px solid #4285F4;
                padding-bottom: 10px;
            }

            .recommendation {
                background-color: #e6f4ea;
                border-left: 5px solid #34a853;
                padding: 15px;
                margin: 15px 0;
                border-radius: 0 5px 5px 0;
            }

            .warning {
                background-color: #fef7e0;
                border-left: 5px solid #fbbc04;
                padding: 15px;
                margin: 15px 0;
                border-radius: 0 5px 5px 0;
            }

            button {
                background-color: #4285F4;
                color: white;
                border: none;
                padding: 10px 15px;
                border-radius: 5px;
                cursor: pointer;
                font-size: 16px;
                margin: 10px 5px;
            }

            button:hover {
                background-color: #3367d6;
            }
        </style>
        """

        # Display the custom CSS
        display(HTML(custom_css))

        print("✅ Colab environment setup complete")
        return True

    except Exception as e:
        print(f"Error setting up Colab environment: {e}")
        return False

def display_header(title):
    """Display a styled header in Colab."""
    if not USING_COLAB:
        print(f"\n{title}\n{'=' * len(title)}")
        return

    header_html = f'<div class="interview-header">{title}</div>'
    display(HTML(header_html))

def display_question(question_num, total_questions, question_text):
    """Display a formatted interview question."""
    if not USING_COLAB:
        print(f"\nQuestion {question_num}/{total_questions}: {question_text}")
        return

    question_html = f'''
    <div class="question-box">
        <strong>Question {question_num}/{total_questions}:</strong><br>
        {question_text}
    </div>
    '''
    display(HTML(question_html))

def create_response_area():
    """Create an interactive response area for the user."""
    if not USING_COLAB:
        print("\nPlease type your response below:")
        return

    response_html = '''
    <div class="response-box" id="response-area" contenteditable="true">
        Type your interview response here...
    </div>

    <button onclick="submitResponse()">Submit Response</button>

    <script>
    // Define a function to submit the response
    function submitResponse() {
        var responseText = document.getElementById('response-area').innerText;
        if (responseText.trim() === "" || responseText === "Type your interview response here...") {
            alert("Please provide a response before submitting.");
            return;
        }

        // Send the response back to Python
        google.colab.kernel.invokeFunction('notebook.getResponse', [responseText], {});

        // Disable editing after submission
        document.getElementById('response-area').contentEditable = false;
        document.getElementById('response-area').style.backgroundColor = "#f1f3f4";

        // Change button text to show processing
        var buttons = document.getElementsByTagName('button');
        for (var i = 0; i < buttons.length; i++) {
            buttons[i].disabled = true;
            buttons[i].innerText = "Processing...";
        }
    }
    </script>
    '''
    display(HTML(response_html))

    # Register the callback function to get the response
    from google.colab import output

    response_text = [None]  # Using a list to store the value that will be modified by the callback

    def get_response_callback(response):
        response_text[0] = response

    output.register_callback('notebook.getResponse', get_response_callback)

    # Wait for the response
    while response_text[0] is None:
        time.sleep(0.5)

    return response_text[0]

def display_processing_animation():
    """Display an animation while processing the response."""
    if not USING_COLAB:
        print("\nAnalyzing your response...")
        return

    processing_html = '''
    <div style="text-align: center; margin: 20px 0;">
        <p>Analyzing your response...</p>
        <div style="display: inline-block; position: relative; width: 80px; height: 80px;">
            <div style="position: absolute; border: 4px solid #4285F4; opacity: 1; border-radius: 50%; animation: ripple 1s cubic-bezier(0, 0.2, 0.8, 1) infinite;" ></div>
            <div style="position: absolute; border: 4px solid #4285F4; opacity: 1; border-radius: 50%; animation: ripple 1s cubic-bezier(0, 0.2, 0.8, 1) infinite; animation-delay: -0.5s;" ></div>
        </div>
    </div>

    <style>
    @keyframes ripple {
      0% {
        top: 36px;
        left: 36px;
        width: 0;
        height: 0;
        opacity: 1;
      }
      100% {
        top: 0px;
        left: 0px;
        width: 72px;
        height: 72px;
        opacity: 0;
      }
    }
    </style>
    '''
    display(HTML(processing_html))
    time.sleep(2)  # Simulate processing time

def display_score_bar(score, label, color):
    """Display a score as a progress bar."""
    if not USING_COLAB:
        print(f"{label}: {format_score(score)} {get_score_emoji(score)}")
        return

    # Ensure score is between 0 and 100
    score = max(0, min(100, score))

    # Determine bar color based on score
    if color is None:
        if score >= 80:
            color = "#34a853"  # Green
        elif score >= 60:
            color = "#fbbc04"  # Yellow
        elif score >= 40:
            color = "#fa7b17"  # Orange
        else:
            color = "#ea4335"  # Red

    bar_html = f'''
    <div class="metric-row">
        <span class="metric-label">{label}:</span>
        <span>{format_score(score)} {get_score_emoji(score)}</span>
    </div>
    <div class="progress-container">
        <div class="progress-bar" style="width: {score}%; background-color: {color};">{score}%</div>
    </div>
    '''
    return bar_html

def display_results_card(predictions, question=None):
    """Display evaluation results in a visually appealing card format."""
    if not USING_COLAB:
        # Fall back to the original display_results function
        if 'display_results' in globals():
            display_results(predictions, question)
        else:
            print("\nEvaluation Results:")
            print(f"Technical: {format_score(predictions.get('technical', 50))}")
            print(f"Communication: {format_score(predictions.get('communication', 50))}")
            print(f"Problem Solving: {format_score(predictions.get('problem_solving', 50))}")
            print(f"Cultural Fit: {format_score(predictions.get('cultural_fit', 50))}")
            print(f"Overall: {format_score(predictions.get('overall', 50))}")
        return

    # Safely extract prediction values
    def safe_extract(pred_dict, key):
        if key not in pred_dict:
            return 50.0 if key != 'category' else 'Average'

        value = pred_dict[key]
        if isinstance(value, (list, np.ndarray)) and len(value) > 0:
            return value[0]
        return value

    # Get prediction values
    technical = float(safe_extract(predictions, 'technical'))
    communication = float(safe_extract(predictions, 'communication'))
    problem_solving = float(safe_extract(predictions, 'problem_solving'))
    cultural_fit = float(safe_extract(predictions, 'cultural_fit'))
    overall = float(safe_extract(predictions, 'overall'))
    category = str(safe_extract(predictions, 'category'))

    # Create the results card HTML
    results_html = f'''
    <div class="result-card">
        <h2 style="text-align: center; color: #4285F4; margin-top: 0;">Response Evaluation</h2>

        {display_score_bar(technical, "Technical Proficiency", "#4285F4")}
        {display_score_bar(communication, "Communication Skills", "#34a853")}
        {display_score_bar(problem_solving, "Problem-Solving Approach", "#fbbc04")}
        {display_score_bar(cultural_fit, "Team & Culture Fit", "#ea4335")}

        <div style="border-top: 2px solid #f1f3f4; margin: 15px 0; padding-top: 15px;">
            {display_score_bar(overall, "Overall Performance", None)}
        </div>

        <div class="feedback-box">
            <strong>Performance Category:</strong> {category}<br>
            <p>{get_category_description(category)}</p>
        </div>
    </div>
    '''

    display(HTML(results_html))

def plot_radar_chart(predictions):
    """Create and display a radar chart of the evaluation metrics."""
    if not USING_COLAB:
        return

    # Extract scores
    def safe_extract(pred_dict, key):
        if key not in pred_dict:
            return 50.0

        value = pred_dict[key]
        if isinstance(value, (list, np.ndarray)) and len(value) > 0:
            return value[0]
        return value

    # Get values and normalize to 0-10 scale
    technical = float(safe_extract(predictions, 'technical')) / 10
    communication = float(safe_extract(predictions, 'communication')) / 10
    problem_solving = float(safe_extract(predictions, 'problem_solving')) / 10
    cultural_fit = float(safe_extract(predictions, 'cultural_fit')) / 10
    overall = float(safe_extract(predictions, 'overall')) / 10

    # Categories for radar chart
    categories = ['Technical\nProficiency', 'Communication\nSkills',
                  'Problem-Solving\nApproach', 'Team &\nCulture Fit']

    values = [technical, communication, problem_solving, cultural_fit]

    # Set up the radar chart
    plt.figure(figsize=(10, 8))

    # Set background color
    plt.rcParams['axes.facecolor'] = '#f8f9fa'

    # Create angles for each category
    angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()

    # Make the plot circular
    values = values + [values[0]]
    angles = angles + [angles[0]]
    categories = categories + [categories[0]]

    # Create the plot
    ax = plt.subplot(111, polar=True)

    # Plot and fill the polygon
    ax.plot(angles, values, 'o-', linewidth=2, color='#4285F4', label='Your Scores')
    ax.fill(angles, values, alpha=0.25, color='#4285F4')

    # Add a reference circle at score 7.5 (75%)
    reference_values = [7.5] * (len(categories))
    ax.plot(angles, reference_values, '--', linewidth=1, color='#34a853', alpha=0.5, label='Expert Level (7.5)')

    # Set category labels
    ax.set_thetagrids(np.degrees(angles[:-1]), categories[:-1], fontsize=12)

    # Set radial ticks and labels
    ax.set_rlabel_position(0)
    ax.set_rticks([2.5, 5, 7.5, 10])
    ax.set_rlim(0, 10)

    # Add title and legend
    plt.title(f"Interview Performance Metrics\nOverall Score: {overall:.1f}/10",
              size=16, color='#4285F4', y=1.1)

    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

    plt.tight_layout()
    plt.show()

def display_summary_dashboard(all_results):
    """Display a comprehensive summary dashboard of all interview responses."""
    if not USING_COLAB:
        return

    # Calculate average scores
    avg_technical = 0
    avg_communication = 0
    avg_problem_solving = 0
    avg_cultural_fit = 0
    avg_overall = 0
    categories = []

    for result in all_results:
        predictions = result['predictions']

        # Safely extract values
        def safe_extract(pred_dict, key):
            if key not in pred_dict:
                return 50.0 if key != 'category' else 'Average'

            value = pred_dict[key]
            if isinstance(value, (list, np.ndarray)) and len(value) > 0:
                return value[0]
            return value

        # Extract scores
        technical = float(safe_extract(predictions, 'technical'))
        communication = float(safe_extract(predictions, 'communication'))
        problem_solving = float(safe_extract(predictions, 'problem_solving'))
        cultural_fit = float(safe_extract(predictions, 'cultural_fit'))
        overall = float(safe_extract(predictions, 'overall'))
        category = str(safe_extract(predictions, 'category'))

        # Add to totals
        avg_technical += technical
        avg_communication += communication
        avg_problem_solving += problem_solving
        avg_cultural_fit += cultural_fit
        avg_overall += overall
        categories.append(category)

    # Calculate averages
    n = len(all_results)
    avg_technical /= n
    avg_communication /= n
    avg_problem_solving /= n
    avg_cultural_fit /= n
    avg_overall /= n

    # Display summary header
    display_header("Interview Performance Summary")

    # Create summary HTML
    summary_html = f'''
    <div class="result-card">
        <div class="summary-title">Overall Performance</div>

        {display_score_bar(avg_overall, "Average Overall Score", None)}

        <div style="margin-top: 30px;">
            {display_score_bar(avg_technical, "Technical Proficiency", "#4285F4")}
            {display_score_bar(avg_communication, "Communication Skills", "#34a853")}
            {display_score_bar(avg_problem_solving, "Problem-Solving Approach", "#fbbc04")}
            {display_score_bar(avg_cultural_fit, "Team & Culture Fit", "#ea4335")}
        </div>
    </div>
    '''

    display(HTML(summary_html))

    # Create a bar chart comparing areas
    plt.figure(figsize=(12, 6))
    metrics = ['Technical', 'Communication', 'Problem Solving', 'Cultural Fit', 'Overall']
    values = [avg_technical, avg_communication, avg_problem_solving, avg_cultural_fit, avg_overall]
    colors = ['#4285F4', '#34a853', '#fbbc04', '#ea4335', '#673ab7']

    bars = plt.bar(metrics, values, color=colors)

    # Add values on top of bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{height:.1f}%', ha='center', va='bottom', fontweight='bold')

    plt.ylim(0, 105)
    plt.title('Average Performance by Category', fontsize=16, pad=20)
    plt.ylabel('Score (%)', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()

    # Career advice based on overall performance
    advice_html = '''
    <div class="result-card">
        <div class="summary-title">Key Takeaways and Advice</div>
    '''

    # Find weakest and strongest areas
    areas = {
        "Technical knowledge": avg_technical,
        "Communication clarity": avg_communication,
        "Problem-solving approach": avg_problem_solving,
        "Team and culture fit": avg_cultural_fit
    }

    sorted_areas = sorted(areas.items(), key=lambda x: x[1])
    weakest_area = sorted_areas[0]
    strongest_area = sorted_areas[-1]

    advice_html += f'''
        <div class="recommendation">
            <strong>Your strongest area is {strongest_area[0]} ({format_score(strongest_area[1])}).</strong><br>
            Continue to leverage this strength in interviews. Share specific examples that highlight your capabilities in this area.
        </div>

        <div class="warning">
            <strong>Focus on improving {weakest_area[0]} ({format_score(weakest_area[1])}).</strong><br>
            Consider preparing specific examples and practicing responses related to this area before your next interview.
        </div>
    '''

    if avg_overall >= 80:
        advice_html += '''
        <div class="recommendation">
            <strong>You show strong interview skills.</strong><br>
            Consider applying for more challenging positions. Your interview performance suggests you're ready for advanced roles.
        </div>
        '''
    elif avg_overall >= 60:
        advice_html += '''
        <div class="recommendation">
            <strong>You have a good foundation.</strong><br>
            Practice more mock interviews to boost confidence. Focus on providing more specific examples in your answers.
        </div>
        '''
    else:
        advice_html += '''
        <div class="warning">
            <strong>Continue practicing interview scenarios.</strong><br>
            Focus on your weaker areas and consider recording yourself to review your communication style and body language.
        </div>
        '''

    advice_html += '''
        <div style="margin-top: 30px;">
            <strong>Recommended next steps:</strong>
            <ol>
                <li>Review and practice questions in your weaker areas</li>
                <li>Record yourself answering questions to analyze your communication</li>
                <li>Seek feedback from industry professionals if possible</li>
                <li>Consider another practice session focusing on your weakest topics</li>
            </ol>
        </div>
    </div>
    '''

    display(HTML(advice_html))

    # Question breakdown
    question_html = '''
    <div class="result-card">
        <div class="summary-title">Question Breakdown</div>
        <table style="width: 100%; border-collapse: collapse;">
            <tr style="background-color: #4285F4; color: white;">
                <th style="padding: 10px; text-align: left;">Question</th>
                <th style="padding: 10px; text-align: center;">Overall Score</th>
                <th style="padding: 10px; text-align: center;">Category</th>
            </tr>
    '''

    for i, result in enumerate(all_results):
        predictions = result['predictions']
        question = result['question']
        overall = float(safe_extract(predictions, 'overall'))
        category = str(safe_extract(predictions, 'category'))

        # Alternate row colors
        bg_color = "#f8f9fa" if i % 2 == 0 else "white"

        question_html += f'''
            <tr style="background-color: {bg_color};">
                <td style="padding: 10px; border-bottom: 1px solid #dadce0;">{question}</td>
                <td style="padding: 10px; text-align: center; border-bottom: 1px solid #dadce0;">{format_score(overall)} {get_score_emoji(overall)}</td>
                <td style="padding: 10px; text-align: center; border-bottom: 1px solid #dadce0;">{category}</td>
            </tr>
        '''

    question_html += '''
        </table>
    </div>
    '''

    display(HTML(question_html))

# ============================
# ENHANCED INTERVIEW FUNCTIONS
# ============================

def enhanced_run_interview(max_questions=3, all_results=None, question_num=1):
    """
    Run the interactive interview process with an enhanced Colab UI.

    Args:
        max_questions: Maximum number of questions to ask
        all_results: List to store all predictions
        question_num: Current question number
    """
    # Initialize all_results if this is the first question
    if all_results is None:
        all_results = []

        # Set up Colab environment if first question
        if USING_COLAB:
            setup_colab_environment()

    # Clear the output
    if USING_COLAB:
        clear_output(wait=True)
    else:
        if 'clear_screen' in globals():
            clear_screen()
        else:
            print("\n" * 5)  # Simple fallback

    # Show welcome message on the first question
    if question_num == 1:
        display_header("🤖 AI Interview Evaluation System 🤖")

        welcome_html = f'''
        <div style="text-align: center; margin: 20px 0;">
            <p>Welcome to the enhanced interview evaluation experience!</p>
            <p>I'll ask you {max_questions} interview questions, analyze your responses,
            and provide detailed feedback on your performance.</p>
        </div>
        '''

        if USING_COLAB:
            display(HTML(welcome_html))
        else:
            print(f"Welcome to the AI Interview Evaluation System!")
            print(f"I'll ask you {max_questions} interview questions, analyze your responses,")
            print(f"and provide feedback on your performance.\n")

    # List of interview questions
    questions = [
        "Tell me about yourself and your experience in software engineering.",
        "Describe a challenging project you worked on and how you overcame obstacles.",
        "How do you approach debugging a complex issue in your code?",
        "Explain how you would design a scalable web application architecture.",
        "How do you stay updated with the latest technologies in your field?",
        "Describe a situation where you had to work with a difficult team member.",
        "What's your experience with agile development methodologies?",
        "How would you optimize a slow-performing database query?",
        "Tell me about a time when you had to learn a new technology quickly."
    ]

    # Randomly select a question
    question = random.choice(questions)

    # Display the question
    display_question(question_num, max_questions, question)

    # Get the user's response
    if USING_COLAB:
        response = create_response_area()
    else:
        response = input("Your response: ")

    # Show processing animation
    if USING_COLAB:
        clear_output(wait=True)
        display_processing_animation()
    else:
        print("\nAnalyzing your response...")

    # Extract features from the response
    features = extract_features_from_text(response)

    # Make predictions
    predictions = predict_response_scores(features)

    # Clear the output
    if USING_COLAB:
        clear_output(wait=True)

    # Display results
    if USING_COLAB:
        display_results_card(predictions, question)
        plot_radar_chart(predictions)
    else:
        display_results(predictions, question)

    # Store result
    result = {
        'question': question,
        'response': response,
        'predictions': predictions
    }
    all_results.append(result)

    # Ask if the user wants to continue to the next question
    if question_num < max_questions:
        if USING_COLAB:
            next_question_html = '''
            <div style="margin: 30px 0; text-align: center;">
                <button onclick="continueInterview()">Continue to Next Question</button>
            </div>

            <script>
            function continueInterview() {
                google.colab.kernel.invokeFunction('notebook.continueInterview', [], {});

                // Disable button after click
                var buttons = document.getElementsByTagName('button');
                for (var i = 0; i < buttons.length; i++) {
                    buttons[i].disabled = true;
                    buttons[i].innerText = "Loading...";
                }
            }
            </script>
            '''
            display(HTML(next_question_html))

            # Register the callback function
            from google.colab import output

            continue_interview = [False]

            def continue_interview_callback():
                continue_interview[0] = True

            output.register_callback('notebook.continueInterview', continue_interview_callback)

            # Wait for the user to click the button
            while not continue_interview[0]:
                time.sleep(0.5)

            # Continue to the next question
            return enhanced_run_interview(max_questions, all_results, question_num + 1)
        else:
            continue_interview = input("\nContinue to next question? (y/n): ").lower().strip()
            if continue_interview == 'y':
                return enhanced_run_interview(max_questions, all_results, question_num + 1)
    else:
        # Display final summary
        if USING_COLAB:
            clear_output(wait=True)

        display_summary_dashboard(all_results)

    return all_results

# ============================
# MAIN EXECUTION
# ============================

def main():
    """Main function to run the enhanced interview process."""
    print("Section 10: Enhanced Colab Interview Interface")
    print("=============================================")

    if USING_COLAB:
        print("Running in Google Colab environment")
        setup_colab_environment()
    else:
        print("Not running in Google Colab. Some features will be limited.")

    # Prompt for the number of questions
    if USING_COLAB:
        questions_html = '''
        <div style="text-align: center; margin: 20px 0;">
            <h2>How many interview questions would you like to practice?</h2>
            <input type="number" id="num-questions" min="1" max="10" value="3" style="padding: 10px; font-size: 16px; width: 100px; text-align: center;">
            <button onclick="startInterview()">Start Interview</button>
        </div>

        <script>
        function startInterview() {
            var numQuestions = document.getElementById('num-questions').value;
            google.colab.kernel.invokeFunction('notebook.startInterview', [numQuestions], {});

            // Disable button after click
            var buttons = document.getElementsByTagName('button');
            for (var i = 0; i < buttons.length; i++) {
                buttons[i].disabled = true;
                buttons[i].innerText = "Starting...";
            }
        }
        </script>
        '''
        display(HTML(questions_html))

        # Register the callback function
        from google.colab import output

        num_questions = [3]  # Default

        def start_interview_callback(n):
            try:
                num_questions[0] = int(n)
            except:
                num_questions[0] = 3

        output.register_callback('notebook.startInterview', start_interview_callback)

        # Wait for the user to click the start button
        start_interview = [False]

        def trigger_start():
            start_interview[0] = True

        output.register_callback('notebook.startInterview', trigger_start)

        # Wait for the user to start
        while not start_interview[0]:
            time.sleep(0.5)

        # Start the interview with the selected number of questions
        enhanced_run_interview(max_questions=num_questions[0])
    else:
        # CLI version
        try:
            max_questions = int(input("How many interview questions would you like to practice? (1-10): "))
            max_questions = max(1, min(10, max_questions))  # Ensure between 1 and 10
        except:
            max_questions = 3
            print("Using default: 3 questions")

        enhanced_run_interview(max_questions=max_questions)

# Run the main function when this cell is executed in Colab
main()

Section 10: Enhanced Colab Interview Interface
Running in Google Colab environment


✅ Colab environment setup complete


KeyboardInterrupt: 